<a href="https://colab.research.google.com/github/ajuliasousa/TCC-2025-1/blob/main/TCC_Modelos_Tradicionais_DL_Dataset_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Panorama Geral do Trabalho**

**Título:**

Avaliação Comparativa de Modelos de Machine Learning e Deep Learning para Detecção de Discurso de Ódio com Diferentes Técnicas de Representação e Balanceamento de Dados

**Objetivo Geral:**

Avaliar e comparar o desempenho de diferentes algoritmos de classificação, incluindo modelos tradicionais de Machine Learning (ML) e arquiteturas de Deep Learning (DL), na tarefa de detecção de discurso de ódio em dados textuais, analisando o impacto de distintas abordagens de representação vetorial e técnicas de balanceamento de classes.

**Objetivos Específicos:**

1.	Realizar o pré-processamento e a limpeza de um corpus textual rotulado para discurso de ódio.
2.	Representar os dados textuais por meio de TF-IDF e embeddings (pré-treinados ou ajustados).
3.	Treinar e avaliar modelos de ML com TF-IDF: Naive Bayes, Regressão Logística, SVM, Random Forest, LightGBM, MLP.
4.	Preparar dados com tokenização, vocabulário e padding para modelos DL com embeddings.
5.	Treinar redes profundas: CNN e LSTM com embeddings.
6.	Aplicar diferentes estratégias de balanceamento de dados: oversampling, undersampling e uso de class_weight.
7.	Avaliar os modelos usando métricas: F1-score ponderado, AUC, acurácia.
8.	Comparar sistematicamente o desempenho dos modelos em diferentes cenários e técnicas de representação.
9.	Discutir os impactos do tipo de vetorização e do balanceamento nos resultados obtidos.



### **Dataset 2: Twitter Sentiment Analysis**

**Link:**
https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech/data

Neste projeto, foi utilizado o conjunto de dados disponibilizado pela **Analytics Vidhya** no Kaggle, intitulado Twitter Sentiment Analysis for Hate Speech Detection¹. O objetivo principal deste dataset é fornecer uma base para a **detecção automática de discursos de ódio em publicações do Twitter**, com foco específico em manifestações de cunho **racista ou sexista**.

Cada amostra do conjunto é composta pelo texto completo de um tweet e um rótulo binário associado: '1' indica que o tweet contém discurso de ódio (com conteúdo racista ou sexista), enquanto '0' representa tweets livres desse tipo de conteúdo. Para preservar a privacidade, menções a outros usuários foram substituídas por "@user".

O dataset serve como base para o desenvolvimento e avaliação de modelos de classificação supervisionada, sendo amplamente utilizado em tarefas de **Processamento de Linguagem Natural (PLN)** voltadas à análise de sentimentos e moderação de conteúdo.



### **Bibliotecas e Visão do Dataset**

In [ ]:
# bibliotecas
import pandas as pd
import numpy as np

# drive
from google.colab import drive
drive.mount('/content/drive')

train_file_path = '/content/drive/MyDrive/TCC/Datasets/Twitter hate speech/train_tweets.csv'
test_file_path = '/content/drive/MyDrive/TCC/Datasets/Twitter hate speech/test_tweets.csv'

df_train = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)

Mounted at /content/drive


In [ ]:
print("Dataset de Treino:")
print("Primeiras linhas:")
print(df_train.head())
print("\nInformações:")
df_train.info()
print("\nDescrição estatística:")
print(df_train.describe())
print("\nDistribuição da coluna 'label':")
print(df_train['label'].value_counts(normalize=True) * 100)

print("\n" + "="*50 + "\n")

print("Dataset de Teste:")
print("Primeiras linhas:")
print(df_test.head())
print("\nInformações:")
df_test.info()
print("\nDescrição estatística:")
print(df_test.describe())

Dataset de Treino:
Primeiras linhas:
   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

Informações:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB

Descrição estatística:
                 id         label
count  31962.000000  31962.000000
mean   15981.500000      0.070146
std     9226.778988      0.255397
min        1.000000      0.000000
25%     7991.250000      0.0000

A etapa de estatística descritiva permitiu uma visão inicial da composição e distribuição dos dados utilizados neste projeto. O **dataset de treino** é composto por **31.962 tweets**, cada um identificado por um id, acompanhado de um label binário e o conteúdo textual do tweet.

A variável label é a variável-alvo do modelo e representa a presença (1) ou ausência (0) de discurso de ódio no texto. A análise da distribuição mostra um forte desbalanceamento: **apenas 7% dos tweets são classificados como contendo discurso de ódio**, enquanto os demais 93% não apresentam esse tipo de conteúdo. Esse desequilíbrio reforça a importância de aplicar técnicas de balanceamento durante o treinamento dos modelos.

Já o **dataset de teste** contém **17.197 tweets** e serve exclusivamente para avaliação, não incluindo os rótulos (`label`). Ambos os conjuntos não apresentam valores ausentes, e os identificadores (`id`) variam em um intervalo contínuo. A análise exploratória também revelou que os dados estão bem estruturados e prontos para serem utilizados nas etapas subsequentes de pré-processamento e modelagem.

### **Pré-processamento Textual**

**1. Limpeza com clean_text()**

Função que aplica regras para "limpar" os textos brutos:

    Remove URLs: tira links da internet (ex: http://...).

    Remove menções: elimina nomes de usuários (@usuario).

    Remove hashtags: exclui palavras precedidas de # (ou poderia apenas remover o símbolo).

    Remove caracteres não alfabéticos: exclui números, pontuações e símbolos, mantendo letras e acentuação.

    Converte para minúsculas: uniformiza o texto.

    Remove espaços extras: com strip().

Resultado: uma versão mais "limpa" do tweet.

**2. Tokenização e Processamento Avançado**

Com a função preprocess_text_advanced():

    Tokenização: divide o texto em palavras (tokens) com word_tokenize.

    Remoção de stopwords: elimina palavras muito comuns em inglês (ex: the, and, is) que pouco contribuem para a análise.

    Lematização: reduz as palavras à sua forma base (ex: running vira run, cars vira car), usando o WordNetLemmatizer.

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

# regras de limpeza
def clean_text(text):
    text = re.sub(r'http\S+', '', text) # remove URLs
    text = re.sub(r'@\w+', '', text) # remove menções
    text = re.sub(r'#\w+', '', text) # remove hashtags (ou pode mantê-las sem o #)
    # removendo acentos para simplificar a lematização com WordNetLemmatizer
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A) # remove caracteres não alfabéticos
    text = text.lower() # converte para minúsculas
    text = text.strip()
    return text

df_train['cleaned_tweet'] = df_train['tweet'].apply(clean_text)

df_test['cleaned_tweet'] = df_test['tweet'].apply(clean_text)

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading 'punkt_tab'...")
    nltk.download('punkt_tab')
    print("'punkt_tab' downloaded.")


# incializa lematizador e stop words para inglês
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text_advanced(text):
    if pd.isna(text) or not isinstance(text, str):
        return []
    tokens = word_tokenize(text)
    processed_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalpha()] # Adicionado isalpha() para garantir que são palavras
    return processed_tokens

# aplicação do pré-processamento
df_train['processed_tokens'] = df_train['cleaned_tweet'].apply(preprocess_text_advanced)
df_test['processed_tokens'] = df_test['cleaned_tweet'].apply(preprocess_text_advanced)


print("\nExemplo de tokens processados no dataset de Treino:")
print(df_train[['tweet', 'processed_tokens']].head())

print("\nExemplo de tokens processados no dataset de Teste:")
print(df_test[['tweet', 'processed_tokens']].head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


'punkt_tab' downloaded.

Exemplo de tokens processados no dataset de Treino:
                                               tweet  \
0   @user when a father is dysfunctional and is s...   
1  @user @user thanks for #lyft credit i can't us...   
2                                bihday your majesty   
3  #model   i love u take with u all the time in ...   
4             factsguide: society now    #motivation   

                                    processed_tokens  
0  [father, dysfunctional, selfish, drag, kid, dy...  
1  [thanks, credit, cant, use, cause, dont, offer...  
2                                  [bihday, majesty]  
3                       [love, u, take, u, time, ur]  
4                              [factsguide, society]  

Exemplo de tokens processados no dataset de Teste:
                                               tweet  \
0  #studiolife #aislife #requires #passion #dedic...   
1   @user #white #supremacists want everyone to s...   
2  safe ways to heal your #acne!!   

### **Vetorização com  TF-IDF**

Para transformar os textos dos tweets em uma representação numérica adequada aos algoritmos de aprendizado de máquina, foi utilizado o método de vetorização **TF-IDF (Term Frequency-Inverse Document Frequency).**

Esse processo converte cada tweet em um vetor que reflete a relevância de cada termo no contexto do corpus, penalizando palavras muito frequentes e destacando termos mais informativos. Antes da vetorização, os tokens de cada tweet foram recombinados em strings, já que o `TfidfVectorizer` opera sobre textos contínuos.

A vetorização foi aplicada **separadamente nos conjuntos de treino e teste** para evitar vazamento de dados: o vocabulário foi aprendido apenas a partir dos textos de treino e, em seguida, utilizado para transformar os dados de teste. Como resultado, obteve-se uma matriz esparsa de características com as 5.000 palavras mais relevantes do corpus, gerando uma matriz de tamanho (31.962, 5000) para o conjunto de treino e (17.197, 5000) para o conjunto de teste.

Para fins exploratórios e para permitir testes mais amplos, também foi criada uma matriz TF-IDF combinando os dois conjuntos, resultando em uma estrutura final de (49.159, 5000). Essa vetorização garante uma representação consistente entre os conjuntos e prepara os dados para as etapas subsequentes de modelagem supervisionada.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# --- combina df_train e df_test em um único df ---
df_train_temp = df_train.copy() # Evita SettingWithCopyWarning
df_train_temp = df_train_temp.rename(columns={'label': 'class'})

# aplica o vetorizador APENAS aos textos de treino para aprender o vocabulário
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['processed_tokens'].apply(lambda tokens: ' '.join(tokens)))
X_test_tfidf = tfidf_vectorizer.transform(df_test['processed_tokens'].apply(lambda tokens: ' '.join(tokens)))


print("\nForma da matriz TF-IDF de Treino:")
print(X_train_tfidf.shape)
print("\nForma da matriz TF-IDF de Teste:")
print(X_test_tfidf.shape)

# renomeia coluna 'label' para 'class' no df_train
df_train_renamed = df_train.rename(columns={'label': 'class'})
df_test_placeholder_class = df_test.copy()
df_test_placeholder_class['class'] = -1

# concatena os dataframes
df = pd.concat([df_train_renamed[['processed_tokens', 'class']], df_test_placeholder_class[['processed_tokens', 'class']]], ignore_index=True)

# inicializa o TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_tokens'].apply(lambda tokens: ' '.join(tokens)))

print("\nForma da matriz TF-IDF do dataframe combinado:")
print(tfidf_matrix.shape)


Forma da matriz TF-IDF de Treino:
(31962, 5000)

Forma da matriz TF-IDF de Teste:
(17197, 5000)

Forma da matriz TF-IDF do dataframe combinado:
(49159, 5000)


### **Aplicação em dataset desbalanceado**

#### **Divisão em treino/ teste**

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

X_split = X_train_tfidf # usa a matriz TF-IDF vetorizada do dataset de TREINO original
y_split = df_train['label'] # usa as labels do dataset de TREINO original (coluna 'label')

# Dividir os dados de TREINO originais em conjuntos de TREINO e VALIDAÇÃO
# Chama-se X_train_split, X_val_split, etc. para evitar conflito
# com X_train_tfidf e X_test_tfidf que representam os datasets originais.
# test_size=0.20 significa 20% dos dados de TREINO originais para VALIDAÇÃO
# random_state para reprodutibilidade
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_split, y_split, test_size=0.20, random_state=42, stratify=y_split
)

print("Forma dos dados de treino (X_train_split, y_train_split):", X_train_split.shape, y_train_split.shape)
print("Forma dos dados de validação (X_val_split, y_val_split):", X_val_split.shape, y_val_split.shape)

X_train = X_train_split
X_test = X_val_split
y_train = y_train_split
y_test = y_val_split

print("\nVariáveis para treino/teste dos modelos (split do TREINO original):")
print("Forma de X_train:", X_train.shape, "y_train:", y_train.shape)
print("Forma de X_test:", X_test.shape, "y_test:", y_test.shape)

Forma dos dados de treino (X_train_split, y_train_split): (25569, 5000) (25569,)
Forma dos dados de validação (X_val_split, y_val_split): (6393, 5000) (6393,)

Variáveis para treino/teste dos modelos (split do TREINO original):
Forma de X_train: (25569, 5000) y_train: (25569,)
Forma de X_test: (6393, 5000) y_test: (6393,)


#### **Modelos Tradicionais**

Quatro modelos de classificação supervisionada foram treinados para prever categorias de tweets (discurso de ódio, linguagem ofensiva ou nenhum dos dois), utilizando a matriz TF-IDF como entrada. Os modelos testados foram: **Regressão Logística, Naive Bayes Multinomial, Support Vector Machine (SVM com kernel linear) e Random Forest**. Cada modelo foi ajustado com os dados de treino e avaliado com os dados de teste usando métricas como **acurácia, F1-score (ponderado) e AUC (curva ROC, ponderada)**. Os resultados foram armazenados em um dicionário para facilitar a comparação de desempenho entre os classificadores.

**Regressão Logística**

A Regressão Logística foi utilizada como um modelo linear de base para classificação. Ela estima a probabilidade de um tweet pertencer a cada uma das  classes com base nas palavras mais relevantes (extraídas via TF-IDF). O modelo foi treinado com um número maior de iterações (max_iter=1000) para garantir a convergência, dado o tamanho da matriz. Por oferecer suporte ao método predict_proba, foi possível calcular a métrica AUC ponderada (one-vs-rest), o que fornece uma medida robusta da capacidade do modelo em distinguir entre as classes.

**Naive Bayes Multinomial**

O modelo Naive Bayes Multinomial é especialmente adequado para tarefas de classificação de texto, por assumir que as características (neste caso, palavras) ocorrem de forma independente. Ele é simples, eficiente e frequentemente usado como forte baseline em PLN. No experimento, ele também permitiu a geração de probabilidades de classe (predict_proba), o que possibilitou o cálculo da AUC . Apesar de suas suposições simplificadas, o Naive Bayes costuma ter desempenho competitivo quando os dados estão bem vetorizados.

**Support Vector Machine (SVM)**

O SVM foi utilizado com kernel linear, uma configuração comum e eficaz para dados textuais de alta dimensionalidade, como é o caso da matriz TF-IDF. Foi ativada a opção probability=True para possibilitar o cálculo da AUC, embora isso torne o treinamento mais custoso computacionalmente. O SVM busca encontrar hiperplanos que melhor separam as classes, sendo especialmente útil quando há margens claras entre categorias. Apesar de não ser naturalmente probabilístico, sua robustez o torna uma escolha frequente em tarefas de classificação com múltiplas classes.

**Random Forest**

O modelo Random Forest foi treinado com 100 árvores de decisão, combinando os resultados de várias árvores para aumentar a estabilidade e a precisão da predição. Como um modelo de ensemble, ele lida bem com dados complexos e é menos sensível a overfitting do que uma única árvore. Também oferece suporte a predict_proba, permitindo calcular a AUC ponderada. Sua capacidade de capturar interações não lineares entre os termos dos textos pode ser vantajosa em relação a modelos lineares, especialmente quando o texto contém padrões mais sutis.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
import pandas as pd

results = {}

# 1. Regressão Logística
print("Treinando Regressão Logística...")
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_predictions_proba = lr_model.predict_proba(X_test)[:, 1]

print("Avaliação da Regressão Logística:")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))
print("Weighted F1-score:", f1_score(y_test, lr_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, lr_predictions_proba))


# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, lr_predictions_proba)
}

print("-" * 50)

# 2. Naive Bayes Multinomial
print("Treinando Naive Bayes Multinomial...")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_predictions_proba = nb_model.predict_proba(X_test)[:, 1] # classificação binária

print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))
print("Weighted F1-score:", f1_score(y_test, nb_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, nb_predictions_proba))

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, nb_predictions_proba)
}

print("-" * 50)

# 3. Support Vector Machine (SVM)
print("Treinando SVM (Kernel Linear)...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_predictions_proba = svm_model.predict_proba(X_test)[:, 1]

print("Avaliação do SVM:")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))
print("Weighted F1-score:", f1_score(y_test, svm_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, svm_predictions_proba))

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, svm_predictions_proba)
}

print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_predictions_proba = rf_model.predict_proba(X_test)[:, 1]

print("Avaliação do Random Forest:")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))
print("Weighted F1-score:", f1_score(y_test, rf_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, rf_predictions_proba))

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, rf_predictions_proba)
}

print("\nResultados acumulados até agora:")
print(results)

Treinando Regressão Logística...
Avaliação da Regressão Logística:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5945
           1       0.91      0.24      0.38       448

    accuracy                           0.94      6393
   macro avg       0.93      0.62      0.67      6393
weighted avg       0.94      0.94      0.93      6393

Acurácia:  0.9449397778820585
Weighted F1-score: 0.9294819411928334
AUC: 0.8939741529496577
--------------------------------------------------
Treinando Naive Bayes Multinomial...
Avaliação do Naive Bayes Multinomial:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5945
           1       0.96      0.23      0.38       448

    accuracy                           0.95      6393
   macro avg       0.95      0.62      0.67      6393
weighted avg       0.95      0.95      0.93      6393

Acurácia:  0.9457218833098702
Weighted F1-score: 0.9299568965

In [ ]:
# instalações
!pip install tensorflow

# bibliotecas para a CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
# --- Modelos Avançados com TF-IDF ---

# 5. LightGBM (Gradient Boosting)
!pip install lightgbm

import lightgbm as lgb
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

print("\n" + "="*50)
print("Treinando LightGBM...")

# Para problema binário, num_classes será 2
num_classes = len(np.unique(y_train))

# converte matriz esparsa para o formato LightGBM
# uso de objective='binary' para classificação binária
lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_model.fit(X_train, y_train)
lgb_predictions = lgb_model.predict(X_test)
lgb_predictions_proba = lgb_model.predict_proba(X_test)[:, 1]

print("Avaliação do LightGBM:")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))
print("Weighted F1-score:", f1_score(y_test, lgb_predictions, average='weighted'))

# cálculo de AUC
try:
    # Para AUC em problema binário, use y_test (1D) e as probabilidades da classe positiva (1D)
    lgb_auc_score = roc_auc_score(y_test, lgb_predictions_proba)
    print("AUC:", lgb_auc_score)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    lgb_auc_score = None


# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions, average='weighted'),
    'auc': lgb_auc_score
}


print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score


print("Treinando Rede Neural Densa (MLP)...")


num_classes_mlp = len(np.unique(y_train))

mlp_model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Para classificação binária com saída de probabilidade única, uso de 1 unidade com ativação 'sigmoid'
    Dense(1, activation='sigmoid')
])


# uso de 'binary_crossentropy' para a função de perda com a última camada 'sigmoid'
mlp_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

epochs_mlp = 20
batch_size_mlp = 64


print("\nTreinando MLP...")
history_mlp = mlp_model.fit(X_train, y_train,
                          epochs=epochs_mlp,
                          batch_size=batch_size_mlp,
                          validation_data=(X_test, y_test),
                          verbose=1)


print("\nResumo do Modelo MLP:")
mlp_model.summary()

# avaliação do Modelo MLP
print("\nAvaliando MLP no conjunto de teste...")
loss_mlp, accuracy_mlp = mlp_model.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP no conjunto de teste: {accuracy_mlp:.4f}")

# gera previsões de probabilidade (saída sigmoid é a probabilidade da classe positiva)
mlp_predictions_proba_positive = mlp_model.predict(X_test)

# obtém as classes preditas (limiar 0.5)
mlp_predictions = (mlp_predictions_proba_positive >= 0.5).astype(int).flatten()

print("\nAvaliação completa da MLP:")
print(classification_report(y_test, mlp_predictions))

# cálculo de F1-score ponderado para MLP
mlp_weighted_f1 = f1_score(y_test, mlp_predictions, average='weighted')
print("Weighted F1-score:", mlp_weighted_f1)

# cálculo de AUC (para problema binário)
try:
    mlp_auc_score = roc_auc_score(y_test, mlp_predictions_proba_positive)
    print("AUC:", mlp_auc_score)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    mlp_auc_score = None


# dicionário de resultados
mlp_report = classification_report(y_test, mlp_predictions, output_dict=True)
results['MLP (TF-IDF)'] = {
    'accuracy': accuracy_mlp,
    'precision (macro)': mlp_report['macro avg']['precision'],
    'recall (macro)': mlp_report['macro avg']['recall'],
    'f1-score (macro)': mlp_report['macro avg']['f1-score'],
    'f1-score (weighted)': mlp_weighted_f1,
    'auc': mlp_auc_score
}

print("="*50)


Treinando LightGBM...
[LightGBM] [Info] Number of positive: 1794, number of negative: 23775


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.263833 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28584
[LightGBM] [Info] Number of data points in the train set: 25569, number of used features: 1119
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.070163 -> initscore=-2.584187
[LightGBM] [Info] Start training from score -2.584187


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5945
           1       0.79      0.34      0.48       448

    accuracy                           0.95      6393
   macro avg       0.87      0.67      0.73      6393
weighted avg       0.94      0.95      0.94      6393

Acurácia:  0.9475989363366182
Weighted F1-score: 0.9378357517455937
AUC: 0.8625382974888863
--------------------------------------------------
Treinando Rede Neural Densa (MLP)...

Treinando MLP...


...

 Total params: 3,939,845 (15.03 MB)

 Trainable params: 1,313,281 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,564 (10.02 MB)


Avaliando MLP no conjunto de teste...

Acurácia da MLP no conjunto de teste: 0.9463
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step

Avaliação completa da MLP:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5945
           1       0.64      0.53      0.58       448

    accuracy                           0.95      6393
   macro avg       0.80      0.75      0.78      6393
weighted avg       0.94      0.95      0.94      6393

Weighted F1-score: 0.9439306639414655
AUC: 0.8655735612159077


#### **Modelos de DL**

Para explorar abordagens mais profundas de aprendizado, foi implementada **uma rede neural convolucional (CNN)** voltada para a classificação de textos. Como esse tipo de modelo trabalha melhor com sequências de palavras em vez de vetores TF-IDF, os tweets foram tokenizados e convertidos em **sequências inteiras**, com padding para garantir um comprimento fixo.

O modelo foi construído com uma camada de embedding (para mapear palavras em vetores densos), seguida por uma **camada convolucional 1D** que captura padrões locais no texto e uma **camada de pooling** que extrai as informações mais relevantes. Camadas densas e dropout foram adicionadas para refinar o aprendizado e reduzir overfitting. O modelo foi treinado por 10 épocas e avaliado com base em métricas como acurácia, F1-score e AUC. Essa arquitetura permite capturar melhor a estrutura local e semântica dos textos, sendo especialmente útil para dados curtos e ruidosos como tweets.

In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# --- pré-processamento para CNN/LSTM (Baseado nos datasets ORIGINAIS) ---

max_words = 10000
max_sequence_length = 100

# --- tokenização ---
train_texts = df_train['processed_tokens'].apply(lambda tokens: ' '.join(tokens))
test_texts = df_test['processed_tokens'].apply(lambda tokens: ' '.join(tokens))

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
# seta o tokenizer SOMENTE nos textos de TREINO
tokenizer.fit_on_texts(train_texts)

# converte textos para sequências de inteiros
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# --- Padding das Sequências ---
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded (Treino):")
print(train_padded_sequences[:5])
print("\nForma das sequências padded (Treino):", train_padded_sequences.shape)

print("\nExemplo de sequências padded (Teste):")
print(test_padded_sequences[:5])
print("\nForma das sequências padded (Teste):", test_padded_sequences.shape)


# --- prepara labels para CNN/LSTM ---

label_encoder = LabelEncoder()
# seta o LabelEncoder SOMENTE nas labels de TREINO
encoded_y_train_original = label_encoder.fit_transform(df_train['label'])

print("\nLabels originais de Treino:", df_train['label'].unique())
print("Labels de Treino codificadas:", encoded_y_train_original)
print("Classes conhecidas pelo LabelEncoder:", label_encoder.classes_)

# --- divide dados para CNN/LSTM (usando as sequências e labels codificadas DO TREINO ORIGINAL) ---

X_cnn_train_val = train_padded_sequences
y_cnn_train_val = encoded_y_train_original

# divisão dos dados de TREINO ORIGINAL em conjuntos de TREINO e VALIDAÇÃO para CNN/LSTM
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(
    X_cnn_train_val, y_cnn_train_val, test_size=0.20, random_state=42, stratify=y_cnn_train_val
)

# O dataset de TESTE ORIGINAL (df_test) já está vetorizado/padded em test_padded_sequences.
# nomeado de X_test_cnn_final para clareza.

X_test_cnn_final = test_padded_sequences

print("\nForma dos dados de treino para CNN (split do TREINO ORIGINAL):", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de validação para CNN (split do TREINO ORIGINAL):", X_val_cnn.shape, y_val_cnn.shape)
print("Forma dos dados de teste FINAL para CNN (dataset de TESTE ORIGINAL):", X_test_cnn_final.shape)


# --- constrói o Modelo CNN ---

embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

# como o problema é binário, podemos usar 1 unidade com sigmoid para saída de probabilidade única
num_classes_cnn = len(label_encoder.classes_)


model = Sequential([
    # camada de Embedding: mapeia palavras (índices) para vetores densos
    # input_dim = Vocabulário total (max_words + 1 para OOV ou apenas max_words se oov_token não for contado separadamente)
    # Keras Tokenizer inclui o OOV token no num_words se especificado, então o tamanho é num_words
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # Para simplificar, vamos usar uma única camada Conv e depois MaxPooling.
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),

    # Camada Densa (Fully Connected)
    Dense(128, activation='relu'),
    # Camada de Dropout
    Dropout(0.5),
    # Camada de saída: 1 unidade com ativação 'sigmoid' para classificação binária
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


# --- treino do Modelo CNN ---
epochs = 10
batch_size = 32

print("\nTreinando CNN...")
history = model.fit(X_train_cnn, y_train_cnn,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val_cnn, y_val_cnn),
                    verbose=1)

print("\nResumo do Modelo CNN:")
model.summary()

# --- Avaliação o Modelo CNN (no conjunto de VALIDAÇÃO) ---
print("\nAvaliando CNN no conjunto de validação...")
loss_cnn_val, accuracy_cnn_val = model.evaluate(X_val_cnn, y_val_cnn, verbose=0)

print(f"\nAcurácia da CNN no conjunto de validação: {accuracy_cnn_val:.4f}")

cnn_predictions_proba_positive_val = model.predict(X_val_cnn)

# obtém as classes preditas (limiar 0.5)
cnn_predictions_val = (cnn_predictions_proba_positive_val >= 0.5).astype(int).flatten()
print("\nAvaliação completa da CNN no conjunto de validação:")
print(classification_report(y_val_cnn, cnn_predictions_val))

# cálculo de F1-score ponderado para CNN
cnn_weighted_f1_val = f1_score(y_val_cnn, cnn_predictions_val, average='weighted')
print("Weighted F1-score:", cnn_weighted_f1_val)

# cálculo AUC para CNN (problema binário)
try:
    cnn_auc_score_val = roc_auc_score(y_val_cnn, cnn_predictions_proba_positive_val)
    print("AUC:", cnn_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    cnn_auc_score_val = None


print("="*50)

# dicionário de resultados

cnn_report_val = classification_report(y_val_cnn, cnn_predictions_val, output_dict=True)
results['CNN (Embedding+Seq) Validation'] = {
    'accuracy': accuracy_cnn_val,
    'precision (macro)': cnn_report_val['macro avg']['precision'],
    'recall (macro)': cnn_report_val['macro avg']['recall'],
    'f1-score (macro)': cnn_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': cnn_weighted_f1_val,
    'auc': cnn_auc_score_val
}


Exemplo de sequências padded (Treino):
[[  22 9294 2872 2396  142 6435    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  81 1538   17  254  376   21  836 6436 3621 9295    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


800/800 ━━━━━━━━━━━━━━━━━━━━ 50s 58ms/step - accuracy: 0.9248 - loss: 0.2498 - val_accuracy: 0.9499 - val_loss: 0.1576
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 44s 55ms/step - accuracy: 0.9660 - loss: 0.0986 - val_accuracy: 0.9520 - val_loss: 0.1579
Epoch 3/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 88s 62ms/step - accuracy: 0.9862 - loss: 0.0460 - val_accuracy: 0.9551 - val_loss: 0.2079
Epoch 4/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 46s 58ms/step - accuracy: 0.9944 - loss: 0.0226 - val_accuracy: 0.9485 - val_loss: 0.2629
Epoch 5/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 78s 53ms/step - accuracy: 0.9965 - loss: 0.0132 - val_accuracy: 0.9463 - val_loss: 0.3401
Epoch 6/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 84s 55ms/step - accuracy: 0.9970 - loss: 0.0104 - val_accuracy: 0.9463 - val_loss: 0.3448
Epoch 7/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 45s 56ms/step - accuracy: 0.9978 - loss: 0.0083 - val_accuracy: 0.9476 - val_loss: 0.4247
Epoch 8/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 44s 55ms/step - accuracy: 0.9973 - loss: 0.0091 - val_accurac

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 98, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,037,765 (15.40 MB)

 Trainable params: 1,345,921 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,691,844 (10.27 MB)


Avaliando CNN no conjunto de validação...

Acurácia da CNN no conjunto de validação: 0.9415
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step

Avaliação completa da CNN no conjunto de validação:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      5945
           1       0.59      0.56      0.57       448

    accuracy                           0.94      6393
   macro avg       0.78      0.77      0.77      6393
weighted avg       0.94      0.94      0.94      6393

Weighted F1-score: 0.9408798084526357
AUC: 0.8630799065841643


In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np


# --- reutiliza dados preparados para a CNN ---


# --- constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

# como o problema é binário, a última camada da LSTM deve ter 1 unidade com ativação 'sigmoid'.
num_classes_lstm = len(label_encoder.classes_)

model_lstm = Sequential([
    # camada de Embedding: reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),

    # Camada LSTM
    LSTM(128),

    # Camada de Dropout
    Dropout(0.5),

    # Camada Densa (Fully Connected)
    Dense(64, activation='relu'), # Camada densa adicional
    Dropout(0.5),

    # Camada de saída: 1 unidade com ativação 'sigmoid' para classificação binária
    Dense(1, activation='sigmoid')
])

model_lstm.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])


# --- treino do Modelo LSTM ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM...")

history_lstm = model_lstm.fit(X_train_cnn, y_train_cnn,
                              epochs=epochs_lstm,
                              batch_size=batch_size_lstm,
                              validation_data=(X_val_cnn, y_val_cnn),
                              verbose=1)

print("\nResumo do Modelo LSTM:")
model_lstm.summary()

# --- avaliação do Modelo LSTM (no conjunto de VALIDAÇÃO) ---
print("\nAvaliando LSTM no conjunto de validação...")

loss_lstm_val, accuracy_lstm_val = model_lstm.evaluate(X_val_cnn, y_val_cnn, verbose=0)
print(f"\nAcurácia da LSTM no conjunto de validação: {accuracy_lstm_val:.4f}")

# gera previsões (classes) e probabilidades para calcular métricas
# saída da camada Dense(1, 'sigmoid') é a probabilidade da classe positiva, forma (n_samples, 1)
lstm_predictions_proba_positive_val = model_lstm.predict(X_val_cnn)

# obtém as classes preditas
lstm_predictions_val = (lstm_predictions_proba_positive_val >= 0.5).astype(int).flatten()
print("\nAvaliação completa da LSTM no conjunto de validação:")
print(classification_report(y_val_cnn, lstm_predictions_val))


# cálculo de F1-score ponderado para LSTM
lstm_weighted_f1_val = f1_score(y_val_cnn, lstm_predictions_val, average='weighted')
print("Weighted F1-score:", lstm_weighted_f1_val)

# cálculo de AUC para LSTM (problema binário)
try:
    lstm_auc_score_val = roc_auc_score(y_val_cnn, lstm_predictions_proba_positive_val)
    print("AUC:", lstm_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    lstm_auc_score_val = None


print("="*50)

# dicionário de resultados

lstm_report_val = classification_report(y_val_cnn, lstm_predictions_val, output_dict=True) # Não precisamos de target_names aqui

results['LSTM (Embedding+Seq) Validation'] = {
    'accuracy': accuracy_lstm_val,
    'precision (macro)': lstm_report_val['macro avg']['precision'],
    'recall (macro)': lstm_report_val['macro avg']['recall'],
    'f1-score (macro)': lstm_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': lstm_weighted_f1_val,
    'auc': lstm_auc_score_val
}



Construindo o Modelo LSTM...

Treinando LSTM...
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


800/800 ━━━━━━━━━━━━━━━━━━━━ 255s 314ms/step - accuracy: 0.9276 - loss: 0.2874 - val_accuracy: 0.9299 - val_loss: 0.2546
Epoch 2/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 247s 295ms/step - accuracy: 0.9302 - loss: 0.2656 - val_accuracy: 0.9299 - val_loss: 0.2539
Epoch 3/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 227s 284ms/step - accuracy: 0.9327 - loss: 0.2562 - val_accuracy: 0.9299 - val_loss: 0.2540
Epoch 4/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 266s 288ms/step - accuracy: 0.9306 - loss: 0.2607 - val_accuracy: 0.9299 - val_loss: 0.2548
Epoch 5/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 262s 289ms/step - accuracy: 0.9297 - loss: 0.2616 - val_accuracy: 0.9299 - val_loss: 0.2552
Epoch 6/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 255s 280ms/step - accuracy: 0.9306 - loss: 0.2586 - val_accuracy: 0.9299 - val_loss: 0.2539
Epoch 7/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 228s 285ms/step - accuracy: 0.9283 - loss: 0.2629 - val_accuracy: 0.9299 - val_loss: 0.2590
Epoch 8/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 226s 283ms/step - accuracy: 0.9320 - loss: 0.25

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 1,419,905 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,839,812 (10.83 MB)


Avaliando LSTM no conjunto de validação...

Acurácia da LSTM no conjunto de validação: 0.9299
200/200 ━━━━━━━━━━━━━━━━━━━━ 17s 85ms/step

Avaliação completa da LSTM no conjunto de validação:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5945
           1       0.00      0.00      0.00       448

    accuracy                           0.93      6393
   macro avg       0.46      0.50      0.48      6393
weighted avg       0.86      0.93      0.90      6393

Weighted F1-score: 0.8961572925201334
AUC: 0.5004432371140214


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

### **Balanceamento do dataset e retreino dos modelos**

#### **Class_weight**

Nesta etapa do experimento, foi aplicado o **balanceamento de classes por meio do parâmetro class_weight**, com o objetivo de mitigar o impacto do desbalanceamento dos dados na performance dos modelos.

Utilizando a função `compute_class_weight` da biblioteca `sklearn`, foram calculados pesos proporcionais à frequência das classes no conjunto de treinamento. Esses pesos foram incorporados diretamente ao processo de treinamento em modelos que suportam esse recurso, como **Regressão Logística, SVM, Random Forest, LightGBM e Redes Neurais com Keras**.

A inclusão dos pesos penaliza erros cometidos em classes minoritárias, forçando os algoritmos a considerarem com maior atenção esses exemplos menos frequentes. Essa estratégia é especialmente eficaz em cenários de classificação desbalanceada, como o de detecção de discursos ofensivos, contribuindo para **melhorias em métricas como recall e F1-score das classes minoritárias, sem necessariamente comprometer a acurácia geral**.

Modelos como o Naive Bayes Multinomial, que não oferecem suporte direto a class_weight, foram mantidos como referência sem ajuste neste momento.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np


results = {}

print("Treinando modelos tradicionais COM Class Weight...")
print("-" * 50)

# 1. Regressão Logística
print("Treinando Regressão Logística (Class Weight)...")
lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42) # random_state para reprodutibilidade
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)
lr_predictions_proba = lr_model.predict_proba(X_test)[:, 1]

print("Avaliação da Regressão Logística (Class Weight):")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))
print("Weighted F1-score:", f1_score(y_test, lr_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, lr_predictions_proba))


# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression (Class Weight)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, lr_predictions_proba)
}

print("-" * 50)

# 2. Naive Bayes Multinomial
# Nota: MultinomialNB NÃO tem o parâmetro class_weight.
# Naive Bayes já lida com a frequência das classes inerentemente na sua formulação de probabilidade.
# Aplicar class_weight a ele não é diretamente possível via parâmetro.
# O modelo será treinado sem class_weight como no código original e adicionado ao dicionário

print("Treinando Naive Bayes Multinomial (Não suporta class_weight)...")
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_predictions = nb_model.predict(X_test)
nb_predictions_proba = nb_model.predict_proba(X_test)[:, 1]

print("Avaliação do Naive Bayes Multinomial:")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))
print("Weighted F1-score:", f1_score(y_test, nb_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, nb_predictions_proba))

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, nb_predictions_proba)
}

print("-" * 50)

# 3. Support Vector Machine (SVM)
print("Treinando SVM (Kernel Linear) (Class Weight)...")
svm_model = SVC(kernel='linear', probability=True, class_weight='balanced', random_state=42)
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)
svm_predictions_proba = svm_model.predict_proba(X_test)[:, 1]

print("Avaliação do SVM (Class Weight):")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))
print("Weighted F1-score:", f1_score(y_test, svm_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, svm_predictions_proba))

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM (Class Weight)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, svm_predictions_proba)
}


print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest (Class Weight)...")
rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_predictions_proba = rf_model.predict_proba(X_test)[:, 1]

print("Avaliação do Random Forest (Class Weight):")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))
print("Weighted F1-score:", f1_score(y_test, rf_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, rf_predictions_proba))

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest (Class Weight)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, rf_predictions_proba)
}

print("\nResultados acumulados (incluindo Class Weight) até agora:")
print(results)

Treinando modelos tradicionais COM Class Weight...
--------------------------------------------------
Treinando Regressão Logística (Class Weight)...
Avaliação da Regressão Logística (Class Weight):
              precision    recall  f1-score   support

           0       0.98      0.91      0.94      5945
           1       0.36      0.71      0.48       448

    accuracy                           0.89      6393
   macro avg       0.67      0.81      0.71      6393
weighted avg       0.93      0.89      0.91      6393

Acurácia:  0.8911309244486156
Weighted F1-score: 0.9067862044203401
AUC: 0.8983885017421603
--------------------------------------------------
Treinando Naive Bayes Multinomial (Não suporta class_weight)...
Avaliação do Naive Bayes Multinomial:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      5945
           1       0.96      0.23      0.38       448

    accuracy                           0.95      6393
   macro av

In [ ]:
# --- Modelos Avançados com TF-IDF (com Class Weight) ---

# importação de bibliotecas
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight # Importar a função


# --- cálculo de Class Weights ---
# os pesos das classes baseados na distribuição das classes no conjunto de treino original (y_train)
# 'balanced' ajusta os pesos inversamente proporcionais à frequência da classe.
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

print("\nPesos das classes para TF-IDF (Class Weight):")
print(class_weight_dict)

print("\n" + "="*50)
print("Treinando Modelos Avançados com TF-IDF usando CLASS WEIGHT...")

# 5. LightGBM (Gradient Boosting) (com Class Weight)
print("Treinando LightGBM com Class Weight...")

lgb_model_cw = lgb.LGBMClassifier(objective='binary', random_state=42, class_weight=class_weight_dict)
lgb_model_cw.fit(X_train, y_train)
lgb_predictions_cw = lgb_model_cw.predict(X_test)
lgb_predictions_proba_cw = lgb_model_cw.predict_proba(X_test)[:, 1]

print("Avaliação do LightGBM (Class Weight):")
print(classification_report(y_test, lgb_predictions_cw))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions_cw))
print("Weighted F1-score:", f1_score(y_test, lgb_predictions_cw, average='weighted'))
print("AUC:", roc_auc_score(y_test, lgb_predictions_proba_cw))


# dicionário de resultados
lgb_report_cw = classification_report(y_test, lgb_predictions_cw, output_dict=True)
results['LightGBM (Class Weight)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions_cw),
    'precision (macro)': lgb_report_cw['macro avg']['precision'],
    'recall (macro)': lgb_report_cw['macro avg']['recall'],
    'f1-score (macro)': lgb_report_cw['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions_cw, average='weighted'),
    'auc': roc_auc_score(y_test, lgb_predictions_proba_cw)
}


print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras (com Class Weight)
print("Treinando Rede Neural Densa (MLP) com Class Weight...")

# reutiliza a estrutura do modelo MLP
# o número de classes será 2 para a saída sigmoid
num_classes_mlp_cw = len(np.unique(y_train))
print(f"Número de classes em y_train para MLP (Class Weight): {num_classes_mlp_cw}")

mlp_model_cw = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


mlp_model_cw.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])


# treino do Modelo MLP
epochs_mlp_cw = 20
batch_size_mlp_cw = 64


print("\nTreinando MLP (Class Weight)...")


history_mlp_cw = mlp_model_cw.fit(X_train, y_train,
                                  epochs=epochs_mlp_cw,
                                  batch_size=batch_size_mlp_cw,
                                  validation_data=(X_test, y_test),
                                  class_weight=class_weight_dict,
                                  verbose=1)


print("\nResumo do Modelo MLP (Class Weight):")
mlp_model_cw.summary()

# avaliação do Modelo MLP
print("\nAvaliando MLP (Class Weight) no conjunto de teste...")
loss_mlp_cw, accuracy_mlp_cw = mlp_model_cw.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP (Class Weight) no conjunto de teste: {accuracy_mlp_cw:.4f}")

# gera previsões de probabilidade e classes
mlp_predictions_proba_positive_cw = mlp_model_cw.predict(X_test)
mlp_predictions_cw = (mlp_predictions_proba_positive_cw >= 0.5).astype(int).flatten()

print("\nAvaliação completa da MLP (Class Weight):")
print(classification_report(y_test, mlp_predictions_cw))

mlp_weighted_f1_cw = f1_score(y_test, mlp_predictions_cw, average='weighted')
print("Weighted F1-score:", mlp_weighted_f1_cw)

try:
    mlp_auc_score_cw = roc_auc_score(y_test, mlp_predictions_proba_positive_cw)
    print("AUC:", mlp_auc_score_cw)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    mlp_auc_score_cw = None


# dicionário de resultados
mlp_report_cw = classification_report(y_test, mlp_predictions_cw, output_dict=True)
results['MLP (Class Weight)'] = {
    'accuracy': accuracy_mlp_cw,
    'precision (macro)': mlp_report_cw['macro avg']['precision'],
    'recall (macro)': mlp_report_cw['macro avg']['recall'],
    'f1-score (macro)': mlp_report_cw['macro avg']['f1-score'],
    'f1-score (weighted)': mlp_weighted_f1_cw,
    'auc': mlp_auc_score_cw
}

print("="*50)


Pesos das classes para TF-IDF (Class Weight):
{np.int64(0): np.float64(0.5377287066246057), np.int64(1): np.float64(7.1262541806020065)}

Treinando Modelos Avançados com TF-IDF usando CLASS WEIGHT...
Treinando LightGBM com Class Weight...
[LightGBM] [Info] Number of positive: 1794, number of negative: 23775


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.202329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28584
[LightGBM] [Info] Number of data points in the train set: 25569, number of used features: 1119
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM (Class Weight):
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      5945
           1       0.32      0.65      0.43       448

    accuracy                           0.88      6393
   macro avg       0.64      0.77      0.68      6393
weighted avg       0.93      0.88      0.90      6393

Acurácia:  0.8781479743469419
Weighted F1-score: 0.8963882467427354
AUC: 0.8658388276462814
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com Class Weight...
Número de classes em y_train para MLP (Class Weight): 2


...

 Total params: 3,939,845 (15.03 MB)

 Trainable params: 1,313,281 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,564 (10.02 MB)


Avaliando MLP (Class Weight) no conjunto de teste...

Acurácia da MLP (Class Weight) no conjunto de teste: 0.9385
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

Avaliação completa da MLP (Class Weight):
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      5945
           1       0.56      0.59      0.58       448

    accuracy                           0.94      6393
   macro avg       0.76      0.78      0.77      6393
weighted avg       0.94      0.94      0.94      6393

Weighted F1-score: 0.9394150008513594
AUC: 0.8605885047458848


In [ ]:
# importação de bibliotecas necessárias
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# --- reutiliza dados preparados anteriormente ---

max_words = 10000
max_sequence_length = 100

# --- tokenização ---
train_texts = df_train['processed_tokens'].apply(lambda tokens: ' '.join(tokens))
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# converte textos para sequências de inteiros
train_sequences = tokenizer.texts_to_sequences(train_texts)

# --- padding das Sequências ---
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# --- prepara labels para CNN/LSTM ---
label_encoder = LabelEncoder()
encoded_y_train_original = label_encoder.fit_transform(df_train['label'])

# ---divide dados para CNN/LSTM (usando as sequências e labels codificadas) ---
X_cnn_train_val = train_padded_sequences # Sequências
y_cnn_train_val = encoded_y_train_original # Labels

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(
    X_cnn_train_val, y_cnn_train_val, test_size=0.20, random_state=42, stratify=y_cnn_train_val
)

print("Variável y_train_cnn regenerada.")
print("Forma de y_train_cnn:", y_train_cnn.shape)
print("Primeiros 5 elementos de y_train_cnn:", y_train_cnn[:5])

Variável y_train_cnn regenerada.
Forma de y_train_cnn: (25569,)
Primeiros 5 elementos de y_train_cnn: [0 1 1 0 0]


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# nomes das classes
classes = np.unique(y_train_cnn)

# cálculo dos pesos das classes
# 'balanced' ajusta automaticamente os pesos inversamente proporcionais às frequências das classes
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_cnn)
class_weight_dict = dict(zip(classes, class_weights))

print("Pesos das classes calculados:")
print(class_weight_dict)

Pesos das classes calculados:
{np.int64(0): np.float64(0.5377287066246057), np.int64(1): np.float64(7.1262541806020065)}


In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score # Import f1_score and roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# --- pré-processamento para CNN ---
results = {}

max_words = 10000
max_sequence_length = 100

# --- tokenização ---
train_texts = df_train['processed_tokens'].apply(lambda tokens: ' '.join(tokens))
test_texts = df_test['processed_tokens'].apply(lambda tokens: ' '.join(tokens))

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# converte textos para sequências de inteiros
train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

# --- padding das Sequências ---
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nExemplo de sequências padded (Treino):")
print(train_padded_sequences[:5])
print("\nForma das sequências padded (Treino):", train_padded_sequences.shape)

print("\nExemplo de sequências padded (Teste):")
print(test_padded_sequences[:5])
print("\nForma das sequências padded (Teste):", test_padded_sequences.shape)

# --- prepara labels para CNN/LSTM ---
label_encoder = LabelEncoder()
encoded_y_train_original = label_encoder.fit_transform(df_train['label'])

print("\nLabels originais de Treino:", df_train['label'].unique())
print("Labels de Treino codificadas:", encoded_y_train_original)
print("Classes conhecidas pelo LabelEncoder:", label_encoder.classes_)


# --- divide dados para CNN/LSTM (usando as sequências e labels codificadas) ---
X_cnn_train_val = train_padded_sequences
y_cnn_train_val = encoded_y_train_original

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(
    X_cnn_train_val, y_cnn_train_val, test_size=0.20, random_state=42, stratify=y_cnn_train_val
)

X_test_cnn_final = test_padded_sequences


print("\nForma dos dados de treino para CNN (split do TREINO ORIGINAL):", X_train_cnn.shape, y_train_cnn.shape)
print("Forma dos dados de validação para CNN (split do TREINO ORIGINAL):", X_val_cnn.shape, y_val_cnn.shape)
print("Forma dos dados de teste FINAL para CNN (dataset de TESTE ORIGINAL):", X_test_cnn_final.shape)


# --- constrói o Modelo CNN ---


embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

num_classes_cnn = len(label_encoder.classes_)


model = Sequential([
    # Camada de Embedding: mapeia palavras (índices) para vetores densos
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    # Múltiplas camadas Convolucionais seguidas de GlobalMaxPooling
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    # Camada Densa (Fully Connected)
    Dense(128, activation='relu'),
    # Camada de Dropout
    Dropout(0.5),
    # Camada de saída
    Dense(num_classes_cnn, activation='softmax')
])


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# --- treino do Modelo CNN ---
epochs = 10
batch_size = 32

print("\nTreinando CNN com class_weight...")
history = model.fit(X_train_cnn, y_train_cnn,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    verbose=1,
                    class_weight=class_weight_dict)

print("\nResumo do Modelo CNN:")
model.summary()

# --- avaliação do modelo
print("\nAvaliando CNN no conjunto de validação...")
loss_cnn_val, accuracy_cnn_val = model.evaluate(X_val_cnn, y_val_cnn, verbose=0)

print(f"\nAcurácia da CNN no conjunto de validação: {accuracy_cnn_val:.4f}")

# classification report para CNN
cnn_predictions_val_probs = model.predict(X_val_cnn)
cnn_predictions_val = tf.argmax(cnn_predictions_val_probs, axis=1).numpy()

print("\nAvaliação completa da CNN no conjunto de validação:")
class_names = [str(cls) for cls in label_encoder.classes_]
print(classification_report(y_val_cnn, cnn_predictions_val, target_names=class_names))

cnn_weighted_f1_val = f1_score(y_val_cnn, cnn_predictions_val, average='weighted')
print("Weighted F1-score:", cnn_weighted_f1_val)

try:
    cnn_auc_score_val = roc_auc_score(y_val_cnn, cnn_predictions_val_probs[:, 1])
    print("AUC:", cnn_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    cnn_auc_score_val = None


print("="*50)

# dicionário de resultados

cnn_report_val = classification_report(y_val_cnn, cnn_predictions_val, output_dict=True, target_names=class_names)
results['CNN (Embedding+Seq)'] = {
    'accuracy': accuracy_cnn_val,
    'precision (macro)': cnn_report_val['macro avg']['precision'],
    'recall (macro)': cnn_report_val['macro avg']['recall'],
    'f1-score (macro)': cnn_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': cnn_weighted_f1_val,
    'auc': cnn_auc_score_val
}


Exemplo de sequências padded (Treino):
[[  22 9294 2872 2396  142 6435    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  81 1538   17  254  376   21  836 6436 3621 9295    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Treinando CNN com class_weight...
Epoch 1/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 32s 39ms/step - accuracy: 0.7417 - loss: 0.5572 - val_accuracy: 0.8788 - val_loss: 0.3422
Epoch 2/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.9236 - loss: 0.2170 - val_accuracy: 0.9050 - val_loss: 0.2548
Epoch 3/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.9682 - loss: 0.0961 - val_accuracy: 0.9077 - val_loss: 0.2837
Epoch 4/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - accuracy: 0.9824 - loss: 0.0517 - val_accuracy: 0.9214 - val_loss: 0.3098
Epoch 5/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9871 - loss: 0.0383 - val_accuracy: 0.9237 - val_loss: 0.3663
Epoch 6/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 27s 38ms/step - accuracy: 0.9908 - loss: 0.0272 - val_accuracy: 0.9320 - val_loss: 0.3916
Epoch 7/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.9893 - loss: 0.0311 - val_accuracy: 0.9288 - val_loss: 0.4189
Epoch 8/10
720/720 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step -

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 98, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,038,152 (15.40 MB)

 Trainable params: 1,346,050 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,692,102 (10.27 MB)


Avaliando CNN no conjunto de validação...

Acurácia da CNN no conjunto de validação: 0.8905
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step

Avaliação completa da CNN no conjunto de validação:
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      5945
           1       0.36      0.70      0.47       448

    accuracy                           0.89      6393
   macro avg       0.67      0.80      0.71      6393
weighted avg       0.93      0.89      0.91      6393

Weighted F1-score: 0.9062961205915044
AUC: 0.8799242310464976


In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# --- reutiliza dados preparados para a CNN ---

# --- constrói o Modelo LSTM ---

print("\n" + "="*50)
print("Construindo o Modelo LSTM...")

num_classes_lstm = len(label_encoder.classes_)
if num_classes_lstm != 2:
    print(f"WARNING: Found {num_classes_lstm} classes. Model is configured for binary (2) classes.")


model_lstm = Sequential([
    # camada de Embedding: Reutiliza os mesmos parâmetros da CNN para consistência na base de embedding
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    # camada LSTM
    LSTM(128),
    # Camada de Dropout
    Dropout(0.5),
    # Camada Densa (Fully Connected)
    Dense(64, activation='relu'), # Camada densa adicional
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


model_lstm.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])


# --- treino do Modelo LSTM ---
epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM...")

history_lstm = model_lstm.fit(X_train_cnn, y_train_cnn,
                              epochs=epochs_lstm,
                              batch_size=batch_size_lstm,
                              validation_data=(X_val_cnn, y_val_cnn),
                              class_weight=class_weight_dict,
                              verbose=1)

print("\nResumo do Modelo LSTM:")
model_lstm.summary()

# --- avaliação do modelo
print("\nAvaliando LSTM no conjunto de validação...")

loss_lstm_val, accuracy_lstm_val = model_lstm.evaluate(X_val_cnn, y_val_cnn, verbose=0)
print(f"\nAcurácia da LSTM no conjunto de validação: {accuracy_lstm_val:.4f}")

lstm_predictions_proba_positive_val = model_lstm.predict(X_val_cnn)
lstm_predictions_val = (lstm_predictions_proba_positive_val >= 0.5).astype(int).flatten()

print("\nAvaliação completa da LSTM no conjunto de validação:")
print(classification_report(y_val_cnn, lstm_predictions_val))

# cálculo de F1-score ponderado para LSTM
lstm_weighted_f1_val = f1_score(y_val_cnn, lstm_predictions_val, average='weighted')
print("Weighted F1-score:", lstm_weighted_f1_val)

# cálculo de AUC
try:
    lstm_auc_score_val = roc_auc_score(y_val_cnn, lstm_predictions_proba_positive_val)
    print("AUC:", lstm_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    lstm_auc_score_val = None


print("="*50)


lstm_report_val = classification_report(y_val_cnn, lstm_predictions_val, output_dict=True)
# dicionário de resultados
results['LSTM (Embedding+Seq) Validation'] = {
    'accuracy': accuracy_lstm_val,
    'precision (macro)': lstm_report_val['macro avg']['precision'],
    'recall (macro)': lstm_report_val['macro avg']['recall'],
    'f1-score (macro)': lstm_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': lstm_weighted_f1_val,
    'auc': lstm_auc_score_val
}


Construindo o Modelo LSTM...

Treinando LSTM...
Epoch 1/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 145s 178ms/step - accuracy: 0.4969 - loss: 0.6982 - val_accuracy: 0.0701 - val_loss: 0.7218
Epoch 2/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 212s 191ms/step - accuracy: 0.3160 - loss: 0.7091 - val_accuracy: 0.0701 - val_loss: 0.7011
Epoch 3/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 192s 178ms/step - accuracy: 0.5294 - loss: 0.6965 - val_accuracy: 0.0701 - val_loss: 0.7007
Epoch 4/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 203s 179ms/step - accuracy: 0.5229 - loss: 0.6868 - val_accuracy: 0.9299 - val_loss: 0.6928
Epoch 5/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 211s 190ms/step - accuracy: 0.5502 - loss: 0.6956 - val_accuracy: 0.9299 - val_loss: 0.6841
Epoch 6/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 143s 179ms/step - accuracy: 0.6055 - loss: 0.6979 - val_accuracy: 0.0701 - val_loss: 0.6940
Epoch 7/15
800/800 ━━━━━━━━━━━━━━━━━━━━ 211s 191ms/step - accuracy: 0.4831 - loss: 0.7004 - val_accuracy: 0.0701 - val_loss: 0.6944
Epoch 8/15
800/800 ━━━━━━━━

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,259,717 (16.25 MB)

 Trainable params: 1,419,905 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,839,812 (10.83 MB)


Avaliando LSTM no conjunto de validação...

Acurácia da LSTM no conjunto de validação: 0.0701
200/200 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step

Avaliação completa da LSTM no conjunto de validação:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5945
           1       0.07      1.00      0.13       448

    accuracy                           0.07      6393
   macro avg       0.04      0.50      0.07      6393
weighted avg       0.00      0.07      0.01      6393

Weighted F1-score: 0.0091782890094146
AUC: 0.5


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

#### **Oversampling**

Como segunda abordagem para tratar o desbalanceamento entre as classes, foi utilizado o método de oversampling sintético **SMOTE (Synthetic Minority Over-sampling Technique)**. Essa técnica atua gerando novos exemplos sintéticos para as classes minoritárias com base na interpolação entre amostras reais vizinhas no espaço de atributos, equilibrando a distribuição sem simplesmente duplicar instâncias existentes.

O SMOTE foi aplicado exclusivamente ao conjunto de treino, preservando a distribuição real nos dados de teste. Com isso, as duas classes (0 e 1) passaram a ter exatamente o mesmo número de instâncias (23.775), conforme evidenciado após o reamostramento.

Esse balanceamento favorece o aprendizado de modelos como Regressão Logística, SVM, Random Forest, LightGBM e MLP, aumentando sua capacidade de generalização para exemplos das classes originalmente sub-representadas. A técnica é especialmente vantajosa por evitar overfitting comum em métodos de duplicação, ao mesmo tempo em que permite o uso direto dos classificadores.

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

In [ ]:
print("\nDistribuição das classes no treino (antes do balanceamento):")
print(y_train.value_counts())

# inicializa SMOTE
# random_state para reprodutibilidade
# sampling_strategy='auto' reamostra todas as classes minoritárias para igualar a classe majoritária
smote = SMOTE(sampling_strategy='auto', random_state=42)

# aplica SMOTE apenas nos dados de TREINO
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("\nForma dos dados de treino após SMOTE:", X_train_resampled.shape, y_train_resampled.shape)
print("\nDistribuição das classes no treino (depois do SMOTE):")
print(y_train_resampled.value_counts())


Distribuição das classes no treino (antes do balanceamento):
label
0    23775
1     1794
Name: count, dtype: int64

Forma dos dados de treino após SMOTE: (47550, 5000) (47550,)

Distribuição das classes no treino (depois do SMOTE):
label
0    23775
1    23775
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
import pandas as pd

results = {}

print("Treinando modelos tradicionais COM SMOTE...")
print("-" * 50)

# 1. Regressão Logística
print("Treinando Regressão Logística com SMOTE-balanced data...")
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_resampled, y_train_resampled)
lr_predictions = lr_model.predict(X_test)
lr_predictions_proba = lr_model.predict_proba(X_test)[:, 1]

print("Avaliação da Regressão Logística (SMOTE):")
print(classification_report(y_test, lr_predictions))
print("Acurácia: ", accuracy_score(y_test, lr_predictions))
print("Weighted F1-score:", f1_score(y_test, lr_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, lr_predictions_proba))


# dicionário de resultados
lr_report = classification_report(y_test, lr_predictions, output_dict=True)
results['Logistic Regression (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions),
    'precision (macro)': lr_report['macro avg']['precision'],
    'recall (macro)': lr_report['macro avg']['recall'],
    'f1-score (macro)': lr_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, lr_predictions_proba)
}

print("-" * 50)

# 2. Naive Bayes Multinomial (geralmente bom para dados textuais esparsos como TF-IDF)
print("Treinando Naive Bayes Multinomial com SMOTE-balanced data...")
nb_model = MultinomialNB()
nb_model.fit(X_train_resampled, y_train_resampled)
nb_predictions = nb_model.predict(X_test)
nb_predictions_proba = nb_model.predict_proba(X_test)[:, 1]

print("Avaliação do Naive Bayes Multinomial (SMOTE):")
print(classification_report(y_test, nb_predictions))
print("Acurácia: ", accuracy_score(y_test, nb_predictions))
print("Weighted F1-score:", f1_score(y_test, nb_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, nb_predictions_proba))

# dicionário de resultados
nb_report = classification_report(y_test, nb_predictions, output_dict=True)
results['Multinomial NB (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, nb_predictions),
    'precision (macro)': nb_report['macro avg']['precision'],
    'recall (macro)': nb_report['macro avg']['recall'],
    'f1-score (macro)': nb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, nb_predictions_proba)
}

print("-" * 50)

# 3. Support Vector Machine (SVM)
print("Treinando SVM (Kernel Linear) com SMOTE-balanced data...")
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
svm_predictions = svm_model.predict(X_test)
svm_predictions_proba = svm_model.predict_proba(X_test)[:, 1]

print("Avaliação do SVM (SMOTE):")
print(classification_report(y_test, svm_predictions))
print("Acurácia: ", accuracy_score(y_test, svm_predictions))
print("Weighted F1-score:", f1_score(y_test, svm_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, svm_predictions_proba))

# dicionário de resultados
svm_report = classification_report(y_test, svm_predictions, output_dict=True)
results['SVM (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions),
    'precision (macro)': svm_report['macro avg']['precision'],
    'recall (macro)': svm_report['macro avg']['recall'],
    'f1-score (macro)': svm_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, svm_predictions_proba)
}

print("-" * 50)

# 4. Random Forest Classifier
print("Treinando Random Forest com SMOTE-balanced data...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_resampled, y_train_resampled)
rf_predictions = rf_model.predict(X_test)
rf_predictions_proba = rf_model.predict_proba(X_test)[:, 1]

print("Avaliação do Random Forest (SMOTE):")
print(classification_report(y_test, rf_predictions))
print("Acurácia: ", accuracy_score(y_test, rf_predictions))
print("Weighted F1-score:", f1_score(y_test, rf_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, rf_predictions_proba))

# dicionário de resultados
rf_report = classification_report(y_test, rf_predictions, output_dict=True)
results['Random Forest (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions),
    'precision (macro)': rf_report['macro avg']['precision'],
    'recall (macro)': rf_report['macro avg']['recall'],
    'f1-score (macro)': rf_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, rf_predictions_proba)
}

print("\nResultados acumulados (incluindo SMOTE) até agora:")
print(results)

Treinando modelos tradicionais COM SMOTE...
--------------------------------------------------
Treinando Regressão Logística com SMOTE-balanced data...
Avaliação da Regressão Logística (SMOTE):
              precision    recall  f1-score   support

           0       0.98      0.85      0.91      5945
           1       0.26      0.71      0.39       448

    accuracy                           0.84      6393
   macro avg       0.62      0.78      0.65      6393
weighted avg       0.93      0.84      0.87      6393

Acurácia:  0.8413890192397935
Weighted F1-score: 0.8722965095263364
AUC: 0.8772295145981016
--------------------------------------------------
Treinando Naive Bayes Multinomial com SMOTE-balanced data...
Avaliação do Naive Bayes Multinomial (SMOTE):
              precision    recall  f1-score   support

           0       0.98      0.89      0.93      5945
           1       0.34      0.72      0.46       448

    accuracy                           0.88      6393
   macro av

In [ ]:
# --- Modelos Avançados com TF-IDF ---

# 5. LightGBM (Gradient Boosting)
!pip install lightgbm

import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np

print("\n" + "="*50)
print("Treinando LightGBM com SMOTE-balanced data...")

num_classes_lgbm = len(np.unique(y_train_resampled))
print(f"Número de classes em y_train_resampled para LightGBM: {num_classes_lgbm}")

lgb_model = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_model.fit(X_train_resampled, y_train_resampled)
lgb_predictions = lgb_model.predict(X_test)
lgb_predictions_proba = lgb_model.predict_proba(X_test)[:, 1]

print("Avaliação do LightGBM (SMOTE):")
print(classification_report(y_test, lgb_predictions))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions))
print("Weighted F1-score:", f1_score(y_test, lgb_predictions, average='weighted'))
print("AUC:", roc_auc_score(y_test, lgb_predictions_proba))


# dicionário de resultados
lgb_report = classification_report(y_test, lgb_predictions, output_dict=True)
results['LightGBM (SMOTE)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions),
    'precision (macro)': lgb_report['macro avg']['precision'],
    'recall (macro)': lgb_report['macro avg']['recall'],
    'f1-score (macro)': lgb_report['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions, average='weighted'),
    'auc': roc_auc_score(y_test, lgb_predictions_proba)
}

print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np

print("Treinando Rede Neural Densa (MLP) com SMOTE-balanced data...")

num_classes_mlp_smote = len(np.unique(y_train_resampled))
print(f"Número de classes em y_train_resampled para MLP: {num_classes_mlp_smote}")

# constrói o Modelo MLP
mlp_model_smote = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

mlp_model_smote.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])


# treino do Modelo MLP
epochs_mlp_smote = 20
batch_size_mlp_smote = 64


print("\nTreinando MLP (SMOTE)...")
history_mlp_smote = mlp_model_smote.fit(X_train_resampled, y_train_resampled,
                          epochs=epochs_mlp_smote,
                          batch_size=batch_size_mlp_smote,
                          validation_data=(X_test, y_test),
                          verbose=1)


print("\nResumo do Modelo MLP (SMOTE):")
mlp_model_smote.summary()

# avaliação do modelo
print("\nAvaliando MLP (SMOTE) no conjunto de teste...")
loss_mlp_smote, accuracy_mlp_smote = mlp_model_smote.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP (SMOTE) no conjunto de teste: {accuracy_mlp_smote:.4f}")

mlp_predictions_proba_positive_smote = mlp_model_smote.predict(X_test)
mlp_predictions_smote = (mlp_predictions_proba_positive_smote >= 0.5).astype(int).flatten()

print("\nAvaliação completa da MLP (SMOTE):")
print(classification_report(y_test, mlp_predictions_smote))

# cálculo de F1-score ponderado para MLP (SMOTE)
mlp_weighted_f1_smote = f1_score(y_test, mlp_predictions_smote, average='weighted')
print("Weighted F1-score:", mlp_weighted_f1_smote)

# cálculo de AUC
try:
    mlp_auc_score_smote = roc_auc_score(y_test, mlp_predictions_proba_positive_smote)
    print("AUC:", mlp_auc_score_smote)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    mlp_auc_score_smote = None


# dicionário de resultados
mlp_report_smote = classification_report(y_test, mlp_predictions_smote, output_dict=True)
results['MLP (SMOTE)'] = {
    'accuracy': accuracy_mlp_smote,
    'precision (macro)': mlp_report_smote['macro avg']['precision'],
    'recall (macro)': mlp_report_smote['macro avg']['recall'],
    'f1-score (macro)': mlp_report_smote['macro avg']['f1-score'],
    'f1-score (weighted)': mlp_weighted_f1_smote,
    'auc': mlp_auc_score_smote
}

print("="*50)


Treinando LightGBM com SMOTE-balanced data...
Número de classes em y_train_resampled para LightGBM: 2


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 23775, number of negative: 23775
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 4.319328 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 63019
[LightGBM] [Info] Number of data points in the train set: 47550, number of used features: 2103
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Avaliação do LightGBM (SMOTE):
              precision    recall  f1-score   support

           0       0.97      0.92      0.94      5945
           1       0.37      0.60      0.46       448

    accuracy                           0.90      6393
   macro avg       0.67      0.76      0.70      6393
weighted avg       0.93      0.90      0.91      6393

Acurácia:  0.899734084154544
Weighted F1-score: 0.9104894689727291
AUC: 0.8555636489246665
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com SMOTE-balanced data...
Número de classes em y_train_resampled para MLP: 2

Treinando MLP (SMOTE)...
Epoch 1/20
743/743 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.8146 - loss: 0.3747 - val_accuracy: 0.9143 - val_loss: 0.2597
Epoch 2/20
743/743 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9712 - loss: 0.0772 - val_accuracy: 0.9193 - val_loss: 0.3080
Epoch 3/20
743/743 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9774 - loss: 0.0514 - val_accuracy

...

 Total params: 3,939,845 (15.03 MB)

 Trainable params: 1,313,281 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,564 (10.02 MB)


Avaliando MLP (SMOTE) no conjunto de teste...

Acurácia da MLP (SMOTE) no conjunto de teste: 0.9493
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

Avaliação completa da MLP (SMOTE):
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      5945
           1       0.68      0.52      0.59       448

    accuracy                           0.95      6393
   macro avg       0.82      0.75      0.78      6393
weighted avg       0.94      0.95      0.95      6393

Weighted F1-score: 0.9461434018924282
AUC: 0.8662283731827467


In [ ]:
# reutiliza TensorFlow e bibliotecas de pré-processamento
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, LSTM # Importar LSTM também
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd

# --- pré-processamento para Modelos DL (CNN/LSTM) ---

max_words = 10000
max_sequence_length = 100

train_texts = df_train['processed_tokens'].apply(lambda tokens: ' '.join(tokens))
test_texts = df_test['processed_tokens'].apply(lambda tokens: ' '.join(tokens))

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

# converte textos para sequências de inteiros
train_sequences_original = tokenizer.texts_to_sequences(train_texts)
test_sequences_original = tokenizer.texts_to_sequences(test_texts) # Para avaliação final

# padding das Sequências
train_padded_sequences_original = pad_sequences(train_sequences_original, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded_sequences_original = pad_sequences(test_sequences_original, maxlen=max_sequence_length, padding='post', truncating='post')

print("\nForma das sequências padded (Treino Original):", train_padded_sequences_original.shape)
print("Forma das sequências padded (Teste Original):", test_padded_sequences_original.shape)

# prepara labels
label_encoder = LabelEncoder()
encoded_y_train_original = label_encoder.fit_transform(df_train['label'])

print("\nLabels de Treino codificadas (Originais):", encoded_y_train_original)
print("Classes conhecidas pelo LabelEncoder:", label_encoder.classes_)


# --- divisão em Treino e Validação ---
X_train_pre_oversample, X_val_dl, y_train_pre_oversample, y_val_dl = train_test_split(
    train_padded_sequences_original, encoded_y_train_original, test_size=0.20, random_state=42, stratify=encoded_y_train_original
)

X_test_dl_final = test_padded_sequences_original


print("\nForma dos dados de treino (Pré-Oversampling):", X_train_pre_oversample.shape, y_train_pre_oversample.shape)
print("Forma dos dados de validação:", X_val_dl.shape, y_val_dl.shape)
print("Forma dos dados de teste FINAL:", X_test_dl_final.shape)

# verificação
print("\nDistribuição das classes no conjunto de treino (Pré-Oversampling):")
print(pd.Series(y_train_pre_oversample).value_counts(normalize=True) * 100)


# --- aplicação do Oversampling ---

print("\nAplicando Oversampling (SMOTE) no conjunto de treino...")
smote = SMOTE(random_state=42)

X_train_oversample, y_train_oversample = smote.fit_resample(X_train_pre_oversample, y_train_pre_oversample)

print("\nForma dos dados de treino (Após Oversampling):", X_train_oversample.shape, y_train_oversample.shape)
print("\nDistribuição das classes no conjunto de treino (Após Oversampling):")
print(pd.Series(y_train_oversample).value_counts(normalize=True) * 100)

X_train_dl = X_train_oversample
y_train_dl = y_train_oversample

X_val_dl = X_val_dl
y_val_dl = y_val_dl

# --- construção e treino do Modelo CNN ---

print("\n" + "="*50)
print("Construindo e Treinando o Modelo CNN com Oversampling...")

embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 128

model_cnn_oversample = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_cnn_oversample.compile(optimizer='adam',
                             loss='binary_crossentropy',
                             metrics=['accuracy'])

epochs_cnn = 10
batch_size_cnn = 32

print("\nTreinando CNN com dados oversampleados...")
history_cnn_oversample = model_cnn_oversample.fit(X_train_dl, y_train_dl,
                                                 epochs=epochs_cnn,
                                                 batch_size=batch_size_cnn,
                                                 validation_data=(X_val_dl, y_val_dl),
                                                 verbose=1)

print("\nResumo do Modelo CNN (Oversample):")
model_cnn_oversample.summary()

# --- avaliação do modelo
print("\nAvaliando CNN (Oversample) no conjunto de validação...")
loss_cnn_os_val, accuracy_cnn_os_val = model_cnn_oversample.evaluate(X_val_dl, y_val_dl, verbose=0)

print(f"\nAcurácia da CNN (Oversample) no conjunto de validação: {accuracy_cnn_os_val:.4f}")

cnn_os_predictions_proba_val = model_cnn_oversample.predict(X_val_dl)
cnn_os_predictions_val = (cnn_os_predictions_proba_val >= 0.5).astype(int).flatten()

print("\nAvaliação completa da CNN (Oversample) no conjunto de validação:")
print(classification_report(y_val_dl, cnn_os_predictions_val))

cnn_os_weighted_f1_val = f1_score(y_val_dl, cnn_os_predictions_val, average='weighted')
print("Weighted F1-score:", cnn_os_weighted_f1_val)

try:
    cnn_os_auc_score_val = roc_auc_score(y_val_dl, cnn_os_predictions_proba_val)
    print("AUC:", cnn_os_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    cnn_os_auc_score_val = None

print("="*50)

# dicionário de resultados
cnn_os_report_val = classification_report(y_val_dl, cnn_os_predictions_val, output_dict=True)
results = {}
results['CNN (Embedding+Seq) Oversample Validation'] = {
    'accuracy': accuracy_cnn_os_val,
    'precision (macro)': cnn_os_report_val['macro avg']['precision'],
    'recall (macro)': cnn_os_report_val['macro avg']['recall'],
    'f1-score (macro)': cnn_os_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': cnn_os_weighted_f1_val,
    'auc': cnn_os_auc_score_val
}


Forma das sequências padded (Treino Original): (31962, 100)
Forma das sequências padded (Teste Original): (17197, 100)

Labels de Treino codificadas (Originais): [0 0 0 ... 0 1 0]
Classes conhecidas pelo LabelEncoder: [0 1]

Forma dos dados de treino (Pré-Oversampling): (25569, 100) (25569,)
Forma dos dados de validação: (6393, 100) (6393,)
Forma dos dados de teste FINAL: (17197, 100)

Distribuição das classes no conjunto de treino (Pré-Oversampling):
0    92.983691
1     7.016309
Name: proportion, dtype: float64

Aplicando Oversampling (SMOTE) no conjunto de treino...

Forma dos dados de treino (Após Oversampling): (47550, 100) (47550,)

Distribuição das classes no conjunto de treino (Após Oversampling):
0    50.0
1    50.0
Name: proportion, dtype: float64

Construindo e Treinando o Modelo CNN com Oversampling...

Treinando CNN com dados oversampleados...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1486/1486 ━━━━━━━━━━━━━━━━━━━━ 60s 39ms/step - accuracy: 0.6959 - loss: 0.5670 - val_accuracy: 0.8281 - val_loss: 0.4218
Epoch 2/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 61s 41ms/step - accuracy: 0.8687 - loss: 0.3090 - val_accuracy: 0.7866 - val_loss: 0.4938
Epoch 3/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 85s 43ms/step - accuracy: 0.9446 - loss: 0.1436 - val_accuracy: 0.7923 - val_loss: 0.6299
Epoch 4/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 76s 39ms/step - accuracy: 0.9682 - loss: 0.0818 - val_accuracy: 0.7877 - val_loss: 0.8746
Epoch 5/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 83s 40ms/step - accuracy: 0.9747 - loss: 0.0613 - val_accuracy: 0.7765 - val_loss: 0.9993
Epoch 6/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 82s 40ms/step - accuracy: 0.9801 - loss: 0.0510 - val_accuracy: 0.7918 - val_loss: 0.9323
Epoch 7/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 81s 39ms/step - accuracy: 0.9817 - loss: 0.0457 - val_accuracy: 0.7651 - val_loss: 1.2621
Epoch 8/10
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 81s 39ms/step - accuracy: 0.9826 - loss: 0.04

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 98, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,037,765 (15.40 MB)

 Trainable params: 1,345,921 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,691,844 (10.27 MB)


Avaliando CNN (Oversample) no conjunto de validação...

Acurácia da CNN (Oversample) no conjunto de validação: 0.7640
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step

Avaliação completa da CNN (Oversample) no conjunto de validação:
              precision    recall  f1-score   support

           0       0.97      0.77      0.86      5945
           1       0.19      0.72      0.30       448

    accuracy                           0.76      6393
   macro avg       0.58      0.74      0.58      6393
weighted avg       0.92      0.76      0.82      6393

Weighted F1-score: 0.8189128757635127
AUC: 0.8038445797789259


In [ ]:
# --- construção e treino do Modelo LSTM ---
print("\n" + "="*50)
print("Construindo e Treinando o Modelo LSTM com Oversampling...")

# reutiliza parâmetros de embedding e sequência

model_lstm_oversample = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_lstm_oversample.compile(optimizer='adam',
                              loss='binary_crossentropy',
                              metrics=['accuracy'])

epochs_lstm = 15
batch_size_lstm = 32

print("\nTreinando LSTM com dados oversampleados...")
history_lstm_oversample = model_lstm_oversample.fit(X_train_dl, y_train_dl,
                                                    epochs=epochs_lstm,
                                                    batch_size=batch_size_lstm,
                                                    validation_data=(X_val_dl, y_val_dl),
                                                    verbose=1)

print("\nResumo do Modelo LSTM (Oversample):")
model_lstm_oversample.summary()


# --- avaliação do modelo
print("\nAvaliando LSTM (Oversample) no conjunto de validação...")
loss_lstm_os_val, accuracy_lstm_os_val = model_lstm_oversample.evaluate(X_val_dl, y_val_dl, verbose=0)
print(f"\nAcurácia da LSTM (Oversample) no conjunto de validação: {accuracy_lstm_os_val:.4f}")

lstm_os_predictions_proba_val = model_lstm_oversample.predict(X_val_dl)
lstm_os_predictions_val = (lstm_os_predictions_proba_val >= 0.5).astype(int).flatten()


print("\nAvaliação completa da LSTM (Oversample) no conjunto de validação:")
print(classification_report(y_val_dl, lstm_os_predictions_val))

lstm_os_weighted_f1_val = f1_score(y_val_dl, lstm_os_predictions_val, average='weighted')
print("Weighted F1-score:", lstm_os_weighted_f1_val)

try:
    lstm_os_auc_score_val = roc_auc_score(y_val_dl, lstm_os_predictions_proba_val)
    print("AUC:", lstm_os_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    lstm_os_auc_score_val = None


print("="*50)

# dicionário de resultados
lstm_os_report_val = classification_report(y_val_dl, lstm_os_predictions_val, output_dict=True)
results = {}
results['LSTM (Embedding+Seq) Oversample Validation'] = {
    'accuracy': accuracy_lstm_os_val,
    'precision (macro)': lstm_os_report_val['macro avg']['precision'],
    'recall (macro)': lstm_os_report_val['macro avg']['recall'],
    'f1-score (macro)': lstm_os_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': lstm_os_weighted_f1_val,
    'auc': lstm_os_auc_score_val
}


Construindo e Treinando o Modelo LSTM com Oversampling...

Treinando LSTM com dados oversampleados...
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1486/1486 ━━━━━━━━━━━━━━━━━━━━ 305s 203ms/step - accuracy: 0.4965 - loss: 0.6946 - val_accuracy: 0.0701 - val_loss: 0.7013
Epoch 2/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 313s 197ms/step - accuracy: 0.5025 - loss: 0.6935 - val_accuracy: 0.0701 - val_loss: 0.6974
Epoch 3/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 333s 204ms/step - accuracy: 0.5034 - loss: 0.6932 - val_accuracy: 0.0701 - val_loss: 0.6998
Epoch 4/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 312s 198ms/step - accuracy: 0.4974 - loss: 0.6933 - val_accuracy: 0.0701 - val_loss: 0.7023
Epoch 5/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 329s 203ms/step - accuracy: 0.4954 - loss: 0.6934 - val_accuracy: 0.0701 - val_loss: 0.6968
Epoch 6/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 306s 206ms/step - accuracy: 0.5010 - loss: 0.6932 - val_accuracy: 0.9299 - val_loss: 0.6888
Epoch 7/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 316s 203ms/step - accuracy: 0.4956 - loss: 0.6932 - val_accuracy: 0.0701 - val_loss: 0.6960
Epoch 8/15
1486/1486 ━━━━━━━━━━━━━━━━━━━━ 315s 198ms/step - accuracy: 0.4

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 1,419,905 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,839,812 (10.83 MB)


Avaliando LSTM (Oversample) no conjunto de validação...

Acurácia da LSTM (Oversample) no conjunto de validação: 0.0701
200/200 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step

Avaliação completa da LSTM (Oversample) no conjunto de validação:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      5945
           1       0.07      1.00      0.13       448

    accuracy                           0.07      6393
   macro avg       0.04      0.50      0.07      6393
weighted avg       0.00      0.07      0.01      6393

Weighted F1-score: 0.0091782890094146
AUC: 0.5000841042893187


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

#### **Undersampling**

Como terceira estratégia de balanceamento, foi empregado o método de undersampling aleatório, utilizando a técnica **RandomUnderSampler** da biblioteca `imbalanced-learn`. Diferente do SMOTE, que gera novas amostras para as classes minoritárias, o undersampling reduz o número de instâncias da classe majoritária, removendo exemplos aleatórios até que todas as classes tenham o mesmo número de ocorrências.

No experimento, o balanceamento foi aplicado apenas sobre o conjunto de treino, **igualando as classes 0 e 1 com 1.794 exemplos cada** — valor correspondente à classe originalmente menos representada. Essa abordagem visa mitigar o viés dos modelos em favor da classe dominante, porém com o custo potencial de perda de informação relevante.
Apesar disso, ela permite a avaliação do impacto da redução de dados no desempenho de algoritmos como **Regressão Logística, Naive Bayes, SVM, Random Forest, LightGBM e redes neurais densas**, todos treinados com a base reamostrada, possibilitando uma comparação direta com os resultados obtidos nas estratégias anteriores de balanceamento.

In [ ]:
# importação de bibliotecas
!pip install imbalanced-learn

from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

print("\nDistribuição das classes no treino TF-IDF (antes do undersampling):")
print(pd.Series(y_train).value_counts())

# inicializa RandomUnderSampler
# random_state para reprodutibilidade
# sampling_strategy='auto' subamostra todas as classes majoritárias para igualar a classe minoritária
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# aplica Undersampling nos dados de treino
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

print("\nForma dos dados de treino TF-IDF após Undersampling:", X_train_rus.shape, y_train_rus.shape)
print("\nDistribuição das classes no treino TF-IDF (depois do Undersampling):")
print(pd.Series(y_train_rus).value_counts())


Distribuição das classes no treino TF-IDF (antes do undersampling):
label
0    23775
1     1794
Name: count, dtype: int64

Forma dos dados de treino TF-IDF após Undersampling: (3588, 5000) (3588,)

Distribuição das classes no treino TF-IDF (depois do Undersampling):
label
0    1794
1    1794
Name: count, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score
import pandas as pd

results = {}

print("\n" + "="*50)
print("Treinando modelos tradicionais COM UNDERSAMPLING...")
print("-" * 50)

# 1. Regressão Logística (Undersampling)
print("Treinando Regressão Logística com Undersampling-balanced data...")
lr_model_rus = LogisticRegression(max_iter=1000, random_state=42)
lr_model_rus.fit(X_train_rus, y_train_rus)
lr_predictions_rus = lr_model_rus.predict(X_test)
lr_predictions_proba_rus = lr_model_rus.predict_proba(X_test)[:, 1]

print("Avaliação da Regressão Logística (Undersampling):")
print(classification_report(y_test, lr_predictions_rus))
print("Acurácia: ", accuracy_score(y_test, lr_predictions_rus))
print("Weighted F1-score:", f1_score(y_test, lr_predictions_rus, average='weighted'))
print("AUC:", roc_auc_score(y_test, lr_predictions_proba_rus))

lr_report_rus = classification_report(y_test, lr_predictions_rus, output_dict=True)
results['Logistic Regression (Undersample)'] = {
    'accuracy': accuracy_score(y_test, lr_predictions_rus),
    'precision (macro)': lr_report_rus['macro avg']['precision'],
    'recall (macro)': lr_report_rus['macro avg']['recall'],
    'f1-score (macro)': lr_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lr_predictions_rus, average='weighted'),
    'auc': roc_auc_score(y_test, lr_predictions_proba_rus)
}

print("-" * 50)

# 2. Naive Bayes Multinomial (Undersampling)
print("Treinando Naive Bayes Multinomial com Undersampling-balanced data...")
nb_model_rus = MultinomialNB()
nb_model_rus.fit(X_train_rus, y_train_rus)
nb_predictions_rus = nb_model_rus.predict(X_test)
nb_predictions_proba_rus = nb_model_rus.predict_proba(X_test)[:, 1]

print("Avaliação do Naive Bayes Multinomial (Undersampling):")
print(classification_report(y_test, nb_predictions_rus))
print("Acurácia: ", accuracy_score(y_test, nb_predictions_rus))
print("Weighted F1-score:", f1_score(y_test, nb_predictions_rus, average='weighted'))
print("AUC:", roc_auc_score(y_test, nb_predictions_proba_rus))

nb_report_rus = classification_report(y_test, nb_predictions_rus, output_dict=True)
results['Multinomial NB (Undersample)'] = {
    'accuracy': accuracy_score(y_test, nb_predictions_rus),
    'precision (macro)': nb_report_rus['macro avg']['precision'],
    'recall (macro)': nb_report_rus['macro avg']['recall'],
    'f1-score (macro)': nb_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, nb_predictions_rus, average='weighted'),
    'auc': roc_auc_score(y_test, nb_predictions_proba_rus)
}

print("-" * 50)

# 3. Support Vector Machine (SVM) (Undersampling)
print("Treinando SVM (Kernel Linear) com Undersampling-balanced data...")
svm_model_rus = SVC(kernel='linear', probability=True, random_state=42)
svm_model_rus.fit(X_train_rus, y_train_rus)
svm_predictions_rus = svm_model_rus.predict(X_test)
svm_predictions_proba_rus = svm_model_rus.predict_proba(X_test)[:, 1]

print("Avaliação do SVM (Undersampling):")
print(classification_report(y_test, svm_predictions_rus))
print("Acurácia: ", accuracy_score(y_test, svm_predictions_rus))
print("Weighted F1-score:", f1_score(y_test, svm_predictions_rus, average='weighted'))
print("AUC:", roc_auc_score(y_test, svm_predictions_proba_rus))

svm_report_rus = classification_report(y_test, svm_predictions_rus, output_dict=True)
results['SVM (Undersample)'] = {
    'accuracy': accuracy_score(y_test, svm_predictions_rus),
    'precision (macro)': svm_report_rus['macro avg']['precision'],
    'recall (macro)': svm_report_rus['macro avg']['recall'],
    'f1-score (macro)': svm_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, svm_predictions_rus, average='weighted'),
    'auc': roc_auc_score(y_test, svm_predictions_proba_rus)
}

print("-" * 50)

# 4. Random Forest Classifier (Undersampling)
print("Treinando Random Forest com Undersampling-balanced data...")
rf_model_rus = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_rus.fit(X_train_rus, y_train_rus)
rf_predictions_rus = rf_model_rus.predict(X_test)
rf_predictions_proba_rus = rf_model_rus.predict_proba(X_test)[:, 1]

print("Avaliação do Random Forest (Undersampling):")
print(classification_report(y_test, rf_predictions_rus))
print("Acurácia: ", accuracy_score(y_test, rf_predictions_rus))
print("Weighted F1-score:", f1_score(y_test, rf_predictions_rus, average='weighted'))
print("AUC:", roc_auc_score(y_test, rf_predictions_proba_rus))

rf_report_rus = classification_report(y_test, rf_predictions_rus, output_dict=True)
results['Random Forest (Undersample)'] = {
    'accuracy': accuracy_score(y_test, rf_predictions_rus),
    'precision (macro)': rf_report_rus['macro avg']['precision'],
    'recall (macro)': rf_report_rus['macro avg']['recall'],
    'f1-score (macro)': rf_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, rf_predictions_rus, average='weighted'),
    'auc': roc_auc_score(y_test, rf_predictions_proba_rus)
}

print("\nResultados acumulados (incluindo SMOTE e Undersampling) até agora:")
print(results)


Treinando modelos tradicionais COM UNDERSAMPLING...
--------------------------------------------------
Treinando Regressão Logística com Undersampling-balanced data...
Avaliação da Regressão Logística (Undersampling):
              precision    recall  f1-score   support

           0       0.98      0.85      0.91      5945
           1       0.27      0.74      0.39       448

    accuracy                           0.84      6393
   macro avg       0.62      0.79      0.65      6393
weighted avg       0.93      0.84      0.87      6393

Acurácia:  0.8391991240419209
Weighted F1-score: 0.8712894020616981
AUC: 0.8765397843325725
--------------------------------------------------
Treinando Naive Bayes Multinomial com Undersampling-balanced data...
Avaliação do Naive Bayes Multinomial (Undersampling):
              precision    recall  f1-score   support

           0       0.98      0.81      0.89      5945
           1       0.23      0.78      0.36       448

    accuracy            

In [ ]:
# --- Modelos Avançados com TF-IDF (Undersampling) ---

# 5. LightGBM (Gradient Boosting) (Undersampling)
import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np

print("\n" + "="*50)
print("Treinando LightGBM com Undersampling-balanced data...")

# o número de classes será 2
num_classes_lgbm_rus = len(np.unique(y_train_rus))
print(f"Número de classes em y_train_rus para LightGBM: {num_classes_lgbm_rus}")

lgb_model_rus = lgb.LGBMClassifier(objective='binary', random_state=42)
lgb_model_rus.fit(X_train_rus, y_train_rus)
lgb_predictions_rus = lgb_model_rus.predict(X_test)
lgb_predictions_proba_rus = lgb_model_rus.predict_proba(X_test)[:, 1]

print("Avaliação do LightGBM (Undersampling):")
print(classification_report(y_test, lgb_predictions_rus))
print("Acurácia: ", accuracy_score(y_test, lgb_predictions_rus))
print("Weighted F1-score:", f1_score(y_test, lgb_predictions_rus, average='weighted'))
print("AUC:", roc_auc_score(y_test, lgb_predictions_proba_rus))

lgb_report_rus = classification_report(y_test, lgb_predictions_rus, output_dict=True)
results['LightGBM (Undersample)'] = {
    'accuracy': accuracy_score(y_test, lgb_predictions_rus),
    'precision (macro)': lgb_report_rus['macro avg']['precision'],
    'recall (macro)': lgb_report_rus['macro avg']['recall'],
    'f1-score (macro)': lgb_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': f1_score(y_test, lgb_predictions_rus, average='weighted'),
    'auc': roc_auc_score(y_test, lgb_predictions_proba_rus)
}

print("-" * 50)

# 6. Rede Neural Densa (MLP) com TensorFlow/Keras (Undersampling)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
import numpy as np

print("Treinando Rede Neural Densa (MLP) com Undersampling-balanced data...")

# o número de classes será 2
num_classes_mlp_rus = len(np.unique(y_train_rus))
print(f"Número de classes em y_train_rus para MLP: {num_classes_mlp_rus}")

mlp_model_rus = Sequential([
    Dense(256, activation='relu', input_shape=(X_train_rus.shape[1],)),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') # saída sigmoid para classificação binária
])

mlp_model_rus.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

epochs_mlp_rus = 20
batch_size_mlp_rus = 64

print("\nTreinando MLP (Undersampling)...")
history_mlp_rus = mlp_model_rus.fit(X_train_rus, y_train_rus,
                                  epochs=epochs_mlp_rus,
                                  batch_size=batch_size_mlp_rus,
                                  validation_data=(X_test, y_test),
                                  verbose=1)

print("\nResumo do Modelo MLP (Undersampling):")
mlp_model_rus.summary()

print("\nAvaliando MLP (Undersampling) no conjunto de teste...")
loss_mlp_rus, accuracy_mlp_rus = mlp_model_rus.evaluate(X_test, y_test, verbose=0)

print(f"\nAcurácia da MLP (Undersampling) no conjunto de teste: {accuracy_mlp_rus:.4f}")

mlp_predictions_proba_positive_rus = mlp_model_rus.predict(X_test)
mlp_predictions_rus = (mlp_predictions_proba_positive_rus >= 0.5).astype(int).flatten()

print("\nAvaliação completa da MLP (Undersampling):")
print(classification_report(y_test, mlp_predictions_rus))

mlp_weighted_f1_rus = f1_score(y_test, mlp_predictions_rus, average='weighted')
print("Weighted F1-score:", mlp_weighted_f1_rus)

try:
    mlp_auc_score_rus = roc_auc_score(y_test, mlp_predictions_proba_positive_rus)
    print("AUC:", mlp_auc_score_rus)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    mlp_auc_score_rus = None

mlp_report_rus = classification_report(y_test, mlp_predictions_rus, output_dict=True)
results['MLP (Undersample)'] = {
    'accuracy': accuracy_mlp_rus,
    'precision (macro)': mlp_report_rus['macro avg']['precision'],
    'recall (macro)': mlp_report_rus['macro avg']['recall'],
    'f1-score (macro)': mlp_report_rus['macro avg']['f1-score'],
    'f1-score (weighted)': mlp_weighted_f1_rus,
    'auc': mlp_auc_score_rus
}

print("="*50)


Treinando LightGBM com Undersampling-balanced data...
Número de classes em y_train_rus para LightGBM: 2
[LightGBM] [Info] Number of positive: 1794, number of negative: 1794
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2174
[LightGBM] [Info] Number of data points in the train set: 3588, number of used features: 152
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Avaliação do LightGBM (Undersampling):
              precision    recall  f1-score   support

           0       0.97      0.82      0.89      5945
           1       0.21      0.63      0.32       448

    accuracy                           0.81      6393
   macro avg       0.59      0.73      0.60      6393
weighted avg       0.91      0.81      0.85      6393

Acurácia:  0.8077584858438918
Weighted F1-score: 0.8480370681632683
AUC: 0.8112100880091314
--------------------------------------------------
Treinando Rede Neural Densa (MLP) com Undersampling-balanced data...
Número de classes em y_train_rus para MLP: 2


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Treinando MLP (Undersampling)...
Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - accuracy: 0.5803 - loss: 0.6859 - val_accuracy: 0.8541 - val_loss: 0.5883
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.8337 - loss: 0.4965 - val_accuracy: 0.7209 - val_loss: 0.5340
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.8889 - loss: 0.2685 - val_accuracy: 0.8223 - val_loss: 0.4271
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.9457 - loss: 0.1469 - val_accuracy: 0.8270 - val_loss: 0.4403
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.9667 - loss: 0.0996 - val_accuracy: 0.8143 - val_loss: 0.5135
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.9755 - loss: 0.0749 - val_accuracy: 0.8211 - val_loss: 0.5367
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.9815 - loss: 0.0569 - val_accuracy: 0.8084 - val_loss: 0.6122
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.9856 - los

...

 Total params: 3,939,845 (15.03 MB)

 Trainable params: 1,313,281 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,626,564 (10.02 MB)


Avaliando MLP (Undersampling) no conjunto de teste...

Acurácia da MLP (Undersampling) no conjunto de teste: 0.8137
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

Avaliação completa da MLP (Undersampling):
              precision    recall  f1-score   support

           0       0.98      0.82      0.89      5945
           1       0.24      0.75      0.36       448

    accuracy                           0.81      6393
   macro avg       0.61      0.79      0.63      6393
weighted avg       0.93      0.81      0.85      6393

Weighted F1-score: 0.8538643318115757
AUC: 0.8694502433016941


In [ ]:
# --- undersampling para Modelos DL (CNN/LSTM) ---

# reutiliza RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

print("\nDistribuição das classes no treino DL (Pré-Undersampling):")
print(pd.Series(y_train_pre_oversample).value_counts(normalize=True) * 100)


# inicializa RandomUnderSampler
# random_state para reprodutibilidade
rus_dl = RandomUnderSampler(sampling_strategy='auto', random_state=42)

X_train_undersample_dl, y_train_undersample_dl = rus_dl.fit_resample(X_train_pre_oversample, y_train_pre_oversample)
print("\nForma dos dados de treino DL após Undersampling:", X_train_undersample_dl.shape, y_train_undersample_dl.shape)

# verificação da distribuição
print("\nDistribuição das classes no treino DL (Após Undersampling):")
print(pd.Series(y_train_undersample_dl).value_counts(normalize=True) * 100)


# --- variáveis para treino/validação do Modelos DL com Undersampling ---
X_train_dl_rus = X_train_undersample_dl
y_train_dl_rus = y_train_undersample_dl

X_val_dl = X_val_dl
y_val_dl = y_val_dl


# --- construção e treino do Modelo CNN (agora usando dados subamostrados para treino) ---

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

print("\n" + "="*50)
print("Construindo e Treinando o Modelo CNN com Undersampling...")

# reutiliza parâmetros de embedding e CNN da célula anterior
# max_words, embedding_dim, max_sequence_length, num_filters, filter_sizes

model_cnn_undersample = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=filter_sizes[0], activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_cnn_undersample.compile(optimizer='adam',
                              loss='binary_crossentropy',
                              metrics=['accuracy'])

epochs_cnn_rus = 10
batch_size_cnn_rus = 32

print("\nTreinando CNN com dados subamostrados...")
history_cnn_undersample = model_cnn_undersample.fit(X_train_dl_rus, y_train_dl_rus,
                                                 epochs=epochs_cnn_rus,
                                                 batch_size=batch_size_cnn_rus,
                                                 validation_data=(X_val_dl, y_val_dl),
                                                 verbose=1)

print("\nResumo do Modelo CNN (Undersample):")
model_cnn_undersample.summary()

# --- avaliação do modelo ---
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

print("\nAvaliando CNN (Undersample) no conjunto de validação...")
loss_cnn_us_val, accuracy_cnn_us_val = model_cnn_undersample.evaluate(X_val_dl, y_val_dl, verbose=0)

print(f"\nAcurácia da CNN (Undersample) no conjunto de validação: {accuracy_cnn_us_val:.4f}")

cnn_us_predictions_proba_val = model_cnn_undersample.predict(X_val_dl)
cnn_us_predictions_val = (cnn_us_predictions_proba_val >= 0.5).astype(int).flatten()

print("\nAvaliação completa da CNN (Undersample) no conjunto de validação:")
print(classification_report(y_val_dl, cnn_us_predictions_val))

cnn_us_weighted_f1_val = f1_score(y_val_dl, cnn_us_predictions_val, average='weighted')
print("Weighted F1-score:", cnn_us_weighted_f1_val)

try:
    cnn_us_auc_score_val = roc_auc_score(y_val_dl, cnn_us_predictions_proba_val)
    print("AUC:", cnn_us_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    cnn_us_auc_score_val = None

print("="*50)

# dicionário de resultados
cnn_us_report_val = classification_report(y_val_dl, cnn_us_predictions_val, output_dict=True)
results['CNN (Embedding+Seq) Undersample Validation'] = {
    'accuracy': accuracy_cnn_us_val,
    'precision (macro)': cnn_us_report_val['macro avg']['precision'],
    'recall (macro)': cnn_us_report_val['macro avg']['recall'],
    'f1-score (macro)': cnn_us_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': cnn_us_weighted_f1_val,
    'auc': cnn_us_auc_score_val
}


Distribuição das classes no treino DL (Pré-Undersampling):
0    92.983691
1     7.016309
Name: proportion, dtype: float64

Forma dos dados de treino DL após Undersampling: (3588, 100) (3588,)

Distribuição das classes no treino DL (Após Undersampling):
0    50.0
1    50.0
Name: proportion, dtype: float64

Construindo e Treinando o Modelo CNN com Undersampling...

Treinando CNN com dados subamostrados...
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 66ms/step - accuracy: 0.6025 - loss: 0.6598 - val_accuracy: 0.8577 - val_loss: 0.3910
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 8s 49ms/step - accuracy: 0.8908 - loss: 0.2927 - val_accuracy: 0.7364 - val_loss: 0.5829
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.9633 - loss: 0.1160 - val_accuracy: 0.7511 - val_loss: 0.7000
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 12s 69ms/step - accuracy: 0.9790 - loss: 0.0624 - val_accuracy: 0.8437 - val_loss: 0.4979
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.9912 - loss: 0.0306 - val_accuracy: 0.8439 - val_loss: 0.5611
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.9944 - loss: 0.0203 - val_accuracy: 0.7954 - val_loss: 0.8465
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 9s 49ms/step - accuracy: 0.9909 - loss: 0.0195 - val_accuracy: 0.7879 - val_loss: 0.9246
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 13s 73ms/step - accuracy: 0.9924 - loss: 0.0160 - val_accuracy: 

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 98, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,037,765 (15.40 MB)

 Trainable params: 1,345,921 (5.13 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,691,844 (10.27 MB)


Avaliando CNN (Undersample) no conjunto de validação...

Acurácia da CNN (Undersample) no conjunto de validação: 0.7840
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step

Avaliação completa da CNN (Undersample) no conjunto de validação:
              precision    recall  f1-score   support

           0       0.98      0.78      0.87      5945
           1       0.22      0.80      0.34       448

    accuracy                           0.78      6393
   macro avg       0.60      0.79      0.61      6393
weighted avg       0.93      0.78      0.83      6393

Weighted F1-score: 0.8337273615327018
AUC: 0.8774765709479755


In [ ]:
# --- construção e treino do Modelo LSTM (agora usando dados subamostrados para treino) ---

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

print("\n" + "="*50)
print("Construindo e Treinando o Modelo LSTM com Undersampling...")

# reutiliza parâmetros de embedding e LSTM da célula anterior
# max_words, embedding_dim, max_sequence_length

model_lstm_undersample = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_lstm_undersample.compile(optimizer='adam',
                               loss='binary_crossentropy',
                               metrics=['accuracy'])

epochs_lstm_rus = 15
batch_size_lstm_rus = 32

print("\nTreinando LSTM com dados subamostrados...")
history_lstm_undersample = model_lstm_undersample.fit(X_train_dl_rus, y_train_dl_rus,
                                                   epochs=epochs_lstm_rus,
                                                   batch_size=batch_size_lstm_rus,
                                                   validation_data=(X_val_dl, y_val_dl),
                                                   verbose=1)

print("\nResumo do Modelo LSTM (Undersampling):")
model_lstm_undersample.summary()

# --- avaliação do Modelo LSTM (Undersampling) ---
print("\nAvaliando LSTM (Undersampling) no conjunto de validação...")
loss_lstm_us_val, accuracy_lstm_us_val = model_lstm_undersample.evaluate(X_val_dl, y_val_dl, verbose=0)

print(f"\nAcurácia da LSTM (Undersampling) no conjunto de validação: {accuracy_lstm_us_val:.4f}")

lstm_us_predictions_proba_val = model_lstm_undersample.predict(X_val_dl)
lstm_us_predictions_val = (lstm_us_predictions_proba_val >= 0.5).astype(int).flatten()

print("\nAvaliação completa da LSTM (Undersampling) no conjunto de validação:")
print(classification_report(y_val_dl, lstm_us_predictions_val))

lstm_us_weighted_f1_val = f1_score(y_val_dl, lstm_us_predictions_val, average='weighted')
print("Weighted F1-score:", lstm_us_weighted_f1_val)

try:
    lstm_us_auc_score_val = roc_auc_score(y_val_dl, lstm_us_predictions_proba_val)
    print("AUC:", lstm_us_auc_score_val)
except ValueError as e:
    print(f"Could not calculate AUC: {e}")
    lstm_us_auc_score_val = None

print("="*50)

# dicionário de resultados
lstm_us_report_val = classification_report(y_val_dl, lstm_us_predictions_val, output_dict=True)
results['LSTM (Embedding+Seq) Undersample Validation'] = {
    'accuracy': accuracy_lstm_us_val,
    'precision (macro)': lstm_us_report_val['macro avg']['precision'],
    'recall (macro)': lstm_us_report_val['macro avg']['recall'],
    'f1-score (macro)': lstm_us_report_val['macro avg']['f1-score'],
    'f1-score (weighted)': lstm_us_weighted_f1_val,
    'auc': lstm_us_auc_score_val
}

print("\nResultados Finais Acumulados:")
print(results)


Construindo e Treinando o Modelo LSTM com Undersampling...

Treinando LSTM com dados subamostrados...
Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


113/113 ━━━━━━━━━━━━━━━━━━━━ 39s 316ms/step - accuracy: 0.4868 - loss: 0.6950 - val_accuracy: 0.0701 - val_loss: 0.7040
Epoch 2/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 42s 369ms/step - accuracy: 0.4975 - loss: 0.6953 - val_accuracy: 0.9299 - val_loss: 0.6852
Epoch 3/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 81s 365ms/step - accuracy: 0.4862 - loss: 0.6944 - val_accuracy: 0.9299 - val_loss: 0.6761
Epoch 4/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 83s 373ms/step - accuracy: 0.4940 - loss: 0.6937 - val_accuracy: 0.9299 - val_loss: 0.6849
Epoch 5/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 43s 378ms/step - accuracy: 0.4852 - loss: 0.6945 - val_accuracy: 0.9299 - val_loss: 0.6761
Epoch 6/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 33s 292ms/step - accuracy: 0.4905 - loss: 0.6938 - val_accuracy: 0.9299 - val_loss: 0.6848
Epoch 7/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 51s 378ms/step - accuracy: 0.4843 - loss: 0.6939 - val_accuracy: 0.0701 - val_loss: 0.7409
Epoch 8/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 82s 382ms/step - accuracy: 0.4837 - loss: 0.6951 - val

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 1,419,905 (5.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,839,812 (10.83 MB)


Avaliando LSTM (Undersampling) no conjunto de validação...

Acurácia da LSTM (Undersampling) no conjunto de validação: 0.9299
200/200 ━━━━━━━━━━━━━━━━━━━━ 12s 59ms/step

Avaliação completa da LSTM (Undersampling) no conjunto de validação:
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5945
           1       0.00      0.00      0.00       448

    accuracy                           0.93      6393
   macro avg       0.46      0.50      0.48      6393
weighted avg       0.86      0.93      0.90      6393

Weighted F1-score: 0.8961572925201334
AUC: 0.5000227156674275

Resultados Finais Acumulados:
{'Logistic Regression (Undersample)': {'accuracy': 0.8391991240419209, 'precision (macro)': 0.6224560521003524, 'recall (macro)': 0.7948645695662622, 'f1-score (macro)': 0.6502364468302245, 'f1-score (weighted)': 0.8712894020616981, 'auc': np.float64(0.8765397843325725)}, 'Multinomial NB (Undersample)': {'accuracy': 0.8069763804160801, 'p

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

### **Resultados**

#### **Dataset Original**

| Modelo                        | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)             |
| ----------------------------- | -------- | ------------------ | -------------------------------- |
| Regressão Logística           | 0.9449   | 0.9295             | 0.8940                           |
| Naive Bayes Multinomial       | 0.9457   | 0.9300             | 0.8659                           |
| SVM (Kernel Linear)           | 0.9510   | 0.9409             | 0.8591                           |
| Random Forest                 | 0.9550   | 0.9504             | 0.8849                           |
| LightGBM                      | 0.9476   | 0.9378             | 0.8625                           |
| MLP (Perceptron Multicamadas) | 0.9500   | 0.9439             | 0.8656                           |
| CNN (Convolutional NN)        | 0.9400   | 0.9409             | 0.8631                           |
| LSTM                          | 0.9300   | 0.8962             | 0.5004 *(classe 1 não prevista)* |


#### **Dataset Balanceado**

##### **Quadro Resumo dos Modelos com `class_weight`**

| Modelo                        | Acurácia | F1-score Ponderado    | AUC (ponderado, OVR)             |
| ----------------------------- | -------- | --------------------- | -------------------------------- |
| Regressão Logística           | 0.8911   | 0.9068                | 0.8984                           |
| Naive Bayes Multinomial (\*)  | 0.9457   | 0.9300                | 0.8659                           |
| SVM (Kernel Linear)           | 0.8839   | 0.9013                | 0.8862                           |
| Random Forest                 | 0.9554   | 0.9506                | 0.8904                           |
| LightGBM                      | 0.8781   | 0.8964                | 0.8658                           |
| MLP (Perceptron Multicamadas) | 0.9400   | 0.9394                | 0.8606                           |
| CNN (Convolutional NN)        | 0.8900   | 0.9063  | 0.8799           |
| LSTM                          | 0.0700   | 0.0092                | 0.5000 *(classe 0 não prevista)* |


##### **Quadro Resumo dos Modelos com Oversampling**


| Modelo                        | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)             |
| ----------------------------- | -------- | ------------------ | -------------------------------- |
| Regressão Logística           | 0.8414   | 0.8723             | 0.8772                           |
| Naive Bayes Multinomial       | 0.8816   | 0.9003             | 0.8926                           |
| SVM (Kernel Linear)           | 0.8430   | 0.8729             | 0.8542                           |
| Random Forest                 | 0.9162   | 0.9228             | 0.8840                           |
| LightGBM                      | 0.8997   | 0.9105             | 0.8556                           |
| MLP (Perceptron Multicamadas) | 0.9500   | 0.9461             | 0.8662                           |
| CNN (Convolutional NN)        | 0.7600   | 0.8189             | 0.8038                           |
| LSTM                          | 0.0700   | 0.0092             | 0.5001 *(classe 0 não prevista)* |


##### **Quadro Resumo dos Modelos com Undersampling**

| Modelo                        | Acurácia | F1-score Ponderado | AUC (ponderado, OVR)             |
| ----------------------------- | -------- | ------------------ | -------------------------------- |
| Regressão Logística           | 0.8392   | 0.8713             | 0.8765                           |
| Naive Bayes Multinomial       | 0.8070   | 0.8495             | 0.8800                           |
| SVM (Kernel Linear)           | 0.8201   | 0.8582             | 0.8728                           |
| Random Forest                 | 0.8337   | 0.8676             | 0.8808                           |
| LightGBM                      | 0.8078   | 0.8480             | 0.8112                           |
| MLP (Perceptron Multicamadas) | 0.8100   | 0.8539             | 0.8695                           |
| CNN (Convolutional NN)        | 0.7800   | 0.8337             | 0.8775                           |
| LSTM                          | 0.9300   | 0.8962             | 0.5000 *(classe 1 não prevista)* |
